# PART- 2 `( Handling With Missing Vales )`

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats.mstats import winsorize
import re
from ipywidgets import interact, Dropdown, IntSlider, Output, widgets
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

%matplotlib inline
# %matplotlib notebook

plt.rcParams["figure.figsize"] = (10, 6)
# plt.rcParams['figure.dpi'] = 100

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

pd.options.display.max_rows = 500    # pd.set_option('display.max_rows', None)
pd.options.display.max_columns = 150 # pd.set_option('display.max_columns', None)

In [2]:
df0 = pd.read_csv("autoscout_cleaned.csv")
df = df0.copy()  

In [3]:
df.shape

(2364, 33)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2364 entries, 0 to 2363
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   make_model               2364 non-null   object 
 1   short_description        2362 non-null   object 
 2   make                     2364 non-null   object 
 3   model                    2364 non-null   object 
 4   location                 2364 non-null   object 
 5   price                    2364 non-null   float64
 6   body_type                2364 non-null   object 
 7   type                     2364 non-null   object 
 8   doors                    2364 non-null   float64
 9   warranty                 2364 non-null   object 
 10  mileage                  2364 non-null   float64
 11  first_registration       2364 non-null   object 
 12  gearbox                  2364 non-null   object 
 13  fuel_type                2210 non-null   object 
 14  seller                  

In [5]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

def check_obj_columns(df):
    '''
    Returns NO PROBLEM or column/s which has/have mixed object types.
    '''    
    
    tdf = df.select_dtypes(include=['object']).applymap(type)
    for col in tdf:
        if len(set(tdf[col].values)) > 1:
            print("Column" + " " + color.BOLD + color.RED + col + color.END + " " + "has mixed object types." )
    else:
        if len(set(tdf[col].values)) == 1:
            print(color.BOLD + color.GREEN + " " + "NO PROBLEM" + " " + color.END + " " + "with the data types of Columns in the DataFrame.")

check_obj_columns(df)

Column short_description has mixed object types.
Column fuel_type has mixed object types.
Column drivetrain has mixed object types.
Column comfort_&_convenience has mixed object types.
Column entertainment_&_media has mixed object types.
Column safety_&_security has mixed object types.
Column extras has mixed object types.
Column upholstery has mixed object types.
 NO PROBLEM  with the data types of Columns in the DataFrame.


MISSING VALUES

In [6]:
missing_percentage = df.isnull().sum()*100 / df.shape[0]

LIST COMPREHENSION
 % OF EACH COLUMN IN THE OUTPUT 

In [7]:
miss_val = []
[miss_val.append(column) for column in df.columns if any(df[column].isnull())]

missing_percentage = df[miss_val].isnull().sum()*100 / df[miss_val].shape[0]
missing_percentage = missing_percentage.sort_values(ascending=False)
print("Columns that have missing values :")
print("---"*11)
print(f"{miss_val}\n\n{missing_percentage}")

Columns that have missing values :
---------------------------------
['short_description', 'fuel_type', 'seats', 'engine_size', 'gears', 'drivetrain', 'cylinders', 'comfort_&_convenience', 'entertainment_&_media', 'safety_&_security', 'extras', 'empty_weight', 'upholstery', 'previous_owner']

drivetrain              53.05
cylinders               11.00
gears                   10.49
empty_weight             7.83
previous_owner           6.77
fuel_type                6.51
upholstery               5.67
seats                    1.06
engine_size              0.25
entertainment_&_media    0.21
extras                   0.21
comfort_&_convenience    0.13
safety_&_security        0.13
short_description        0.08
dtype: float64


In [8]:
def df_nans(df, limit):
    """
    Returns a Pandas Series containing the percentage of missing values in each column of the input DataFrame 'df' that
    exceeds the given 'limit'.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame to analyze for missing values.
    limit : float
        The threshold (in percentage) above which to consider a column as having too many missing values.

    Returns:
    --------
    pandas.Series
        A Series containing the percentage of missing values in each column of the input DataFrame 'df' that exceeds
        the given 'limit'.
        If no column has missing values that exceed the given limit, returns a string message indicating this.
    """
    missing = df.isnull().sum()*100 / df.shape[0]
    missing = missing.loc[lambda x : x >= limit]
    if missing.empty:
        return "No columns have missing values that exceed the given limit."
    else:
        return missing

def show_missing_values(limit):
    return df_nans(df, limit)

def column_nans(serial):
    # display percentage of NaNs in a Series
    return serial.isnull().sum()*100 / serial.shape[0]

In [9]:
# the percentage of missing values in each column that exceeds the given 'limit'.

interact(show_missing_values, limit=(0, 100, 5))

interactive(children=(IntSlider(value=50, description='limit', step=5), Output()), _dom_classes=('widget-inter…

<function __main__.show_missing_values(limit)>

DUPLICATION

In [10]:
df.duplicated(keep=False).sum()

317

In [11]:
df[df.duplicated(keep=False)]

,make_model,short_description,make,model,location,price,body_type,type,doors,warranty,mileage,first_registration,gearbox,fuel_type,seller,seats,engine_size,gears,co_emissions,drivetrain,cylinders,comfort_&_convenience,entertainment_&_media,safety_&_security,extras,empty_weight,full_service_history,upholstery,production_date,previous_owner,energy_efficiency_class,new,combined_fuel_cons
79,Mercedes-Benz AMG GT,GT 63 S AMG 4M+ Com/SHD/360/KeyG/Massage/Sitzkil,Mercedes-Benz,AMG GT,"Julius-Rütgers-Str. 1, 15537 Erkner, DE",135670.00,Coupe,Used,5.00,24,74566.00,08/2019,Automatic,Super 95,Dealer,4.00,3982.00,9.00,257.00,4WD,8.00,"['360° camera, Air conditioning, Armrest, Auto...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive headlights, Alarm system, Blin...","Alloy wheels, Ambient lighting, Automatically ...",2120.00,Yes,NaN,2019.00,1.00,unefficient,Mercedes-Benz AMG GT,11.30
80,Mercedes-Benz AMG GT,GT 63 S AMG 4M+ Com/SHD/360/KeyG/Massage/Sitzkil,Mercedes-Benz,AMG GT,"Julius-Rütgers-Str. 1, 15537 Erkner, DE",135670.00,Coupe,Used,5.00,24,74566.00,08/2019,Automatic,Super 95,Dealer,4.00,3982.00,9.00,257.00,4WD,8.00,"['360° camera, Air conditioning, Armrest, Auto...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive headlights, Alarm system, Blin...","Alloy wheels, Ambient lighting, Automatically ...",2120.00,Yes,NaN,2019.00,1.00,unefficient,Mercedes-Benz AMG GT,11.30
83,Mercedes-Benz S 63 AMG,4M+ Pano+DISTRONIC+HUP+TV+ILS+Swarovski,Mercedes-Benz,S 63 AMG,"Wörth 17, 94034 Passau, DE",165850.00,Coupe,Used,3.00,24,26338.00,02/2020,Automatic,Regular/Benzine 91 (Particle filter),Dealer,4.00,3982.00,NaN,254.00,4WD,NaN,"['360° camera, Air conditioning, Automatic cli...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Alarm system, ...","Alloy wheels, Ambient lighting, Automatically ...",2080.00,Yes,Full leather,2020.00,1.00,unefficient,Mercedes-Benz S 63 AMG,11.20
84,Mercedes-Benz S 63 AMG,4M+ Pano+DISTRONIC+HUP+TV+ILS+Swarovski,Mercedes-Benz,S 63 AMG,"Wörth 17, 94034 Passau, DE",165850.00,Coupe,Used,3.00,24,26338.00,02/2020,Automatic,Regular/Benzine 91 (Particle filter),Dealer,4.00,3982.00,NaN,254.00,4WD,NaN,"['360° camera, Air conditioning, Automatic cli...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Alarm system, ...","Alloy wheels, Ambient lighting, Automatically ...",2080.00,Yes,Full leather,2020.00,1.00,unefficient,Mercedes-Benz S 63 AMG,11.20
141,Mercedes-Benz C 43 AMG,"4M Coupé 19""+PANO+MULTIBEAM+BURMESTER",Mercedes-Benz,C 43 AMG,"Altendorfer Straße 44 C, 45127 Essen, DE",67670.00,Coupe,Demonstration,2.00,24,4900.00,05/2022,Automatic,Regular/Benzine 91 (Particle filter),Dealer,4.00,2996.00,NaN,222.00,4WD,6.00,"['Armrest, Automatic climate control, 2 zones,...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive headlights, Central door lock,...","Alloy wheels, Ambient lighting, Automatically ...",1750.00,Yes,Other,2022.00,1.00,unefficient,Mercedes-Benz C 43 AMG,9.70
142,Mercedes-Benz C 43 AMG,"4M Coupé 19""+PANO+MULTIBEAM+BURMESTER",Mercedes-Benz,C 43 AMG,"Altendorfer Straße 44 C, 45127 Essen, DE",67670.00,Coupe,Demonstration,2.00,24,4900.00,05/2022,Automatic,Regular/Benzine 91 (Particle filter),Dealer,4.00,2996.00,NaN,222.00,4WD,6.00,"['Armrest, Automatic climate control, 2 zones,...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive headlights, Central door lock,...","Alloy wheels, Ambient lighting, Automatically ...",1750.00,Yes,Other,2022.00,1.00,unefficient,Mercedes-Benz C 43 AMG,9.70
203,Mercedes-Benz G 400,"d AMG/20""/Multibeam/Burmester/360°K/AHK/",Mercedes-Benz,G 400,"Hallesche Straße 150, 99734 Nordhausen, DE",164710.00,Off-Road/Pick-up,Employee's car,5.00,24,23989.00,04/2021,Automatic,Diesel (Particle filter),Dealer,5.00,2925.00,NaN,281.00,4WD,6.00,"['360° camera, Air conditioning, Cruise contro...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive 

FIVE DEF S FOR FILLING MISSING VALUES

In [12]:
def first_looking(*cols):
    """
    Prints basic information about given columns in a Pandas DataFrame.

    Parameters:
    *cols (str): The names of the columns to analyze.

    Returns:
    None.

    Prints:
    - The name of each column.
    - The percentage of null values in each column.
    - The number of null values in each column.
    - The number of unique values in each column.
    - The value counts for each unique value in each column, including null values.
    """
    for col in cols:
        print("COLUMN NAME    : ", col)
        print("--------------------------------")
        print("per_of_nulls   : ", "%", round(df[col].isnull().sum()*100 / df.shape[0], 2))
        print("num_of_nulls   : ", df[col].isnull().sum())
        print("num_of_uniques : ", df[col].nunique())
        print("--------------------------------")
        print(df[col].value_counts(dropna=False))
        print("\n")

In [13]:
def fill_most_freq(df, group_col, col_name):
    """
    Fills missing values in a column of a Pandas DataFrame `df` with the most frequent value 
    in each group defined by a grouping column.

    Parameters:
    df (pandas.DataFrame): The DataFrame to operate on.
    group_col (str): The name of the grouping column.
    col_name (str): The name of the column to fill missing values in.

    Returns:
    None.

    Prints:
    - The name of the column that was filled.
    - The percentage of null values in the column after filling.
    - The number of null values in the column after filling.
    - The number of unique values in the column after filling.
    - The value counts for each unique value in the column after filling, including null values, sorted by index.
    """    
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    
    print("COLUMN NAME    : ", col_name)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col_name].isnull().sum()*100 / df.shape[0], 2))
    print("num_of_nulls   : ", df[col_name].isnull().sum())
    print("num_of_uniques : ", df[col_name].nunique())
    print("--------------------------------")
    print(df[col_name].value_counts(dropna = False).sort_index())

In [14]:
def fill_mode(df, group_col1, group_col2, col_name):
    """
    Fills missing values in a column of a Pandas DataFrame `df` based on the values in two other columns.

    Parameters:
    df (pandas.DataFrame): The DataFrame to operate on.
    group_col1 (str): The name of the first grouping column.
    group_col2 (str): The name of the second grouping column.
    col_name (str): The name of the column to fill missing values in.

    Returns:
    None.

    Prints:
    - The name of the column that was filled.
    - The percentage of null values in the column after filling.
    - The number of null values in the column after filling.
    - The number of unique values in the column after filling.
    - The value counts for each unique value in the column after filling, including null values.
    """    
    for group1 in list(df[group_col1].unique()):
        for group2 in list(df[group_col2].unique()):
            cond1 = df[group_col1]==group1
            cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
            mode1 = list(df[cond1][col_name].mode())
            mode2 = list(df[cond2][col_name].mode())
            if mode2 != []:
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
            elif mode1 != []:
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
            else:
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])
    
    print("column name    : ", col_name)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col_name].isnull().sum()*100 / df.shape[0], 2))
    print("num_of_nulls   : ", df[col_name].isnull().sum())
    print("num_of_uniques : ", df[col_name].nunique())
    print("--------------------------------")
    print(df[col_name].value_counts(dropna = False))

In [15]:
def fill_prop(df, group_col, col_name):
    """
    Fills missing values in a column of a Pandas DataFrame `df` using forward and backward fill based on a grouping column.

    Parameters:
    df (pandas.DataFrame): The DataFrame to operate on.
    group_col (str): The name of the grouping column.
    col_name (str): The name of the column to fill missing values in.

    Returns:
    None.

    Prints:
    - The name of the column that was filled.
    - The percentage of null values in the column after filling.
    - The number of null values in the column after filling.
    - The number of unique values in the column after filling.
    - The value counts for each unique value in the column after filling, including null values, sorted by index.
    """    
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("COLUMN NAME    : ", col_name)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col_name].isnull().sum()*100 / df.shape[0], 2))
    print("num_of_nulls   : ", df[col_name].isnull().sum())
    print("num_of_uniques : ", df[col_name].nunique())
    print("--------------------------------")
    print(df[col_name].value_counts(dropna = False).sort_index())

In [16]:
def fill(df, group_col1, group_col2, col_name, method):
    """
    Fills missing values in a column of a Pandas DataFrame `df` based on double-stage grouping and a specified filling method.

    Parameters:
    df (pandas.DataFrame): The DataFrame to operate on.
    group_col1 (str): The name of the first grouping column.
    group_col2 (str): The name of the second grouping column.
    col_name (str): The name of the column to fill missing values in.
    method (str): The filling method to use. Can be "mode", "mean", "median", "ffill", or "bfill".

    Returns:
    None.

    Prints:
    None.
    """
    
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])

    elif method == "mean":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df[col_name].mean(), inplace = True)
        
    elif method == "median":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df[col_name].median(), inplace = True)
        
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("COLUMN NAME    : ", col_name)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col_name].isnull().sum()*100 / df.shape[0], 2))
    print("num_of_nulls   : ", df[col_name].isnull().sum())
    print("num_of_uniques : ", df[col_name].nunique())
    print("--------------------------------")
    print(df[col_name].value_counts(dropna = False).sort_index())

MISSING VALUES IN COLUMNS

In [17]:
df_notnull = df[df.columns[~df.isnull().any()]]
df_notnull.columns

Index(['make_model', 'make', 'model', 'location', 'price', 'body_type', 'type',
       'doors', 'warranty', 'mileage', 'first_registration', 'gearbox',
       'seller', 'co_emissions', 'full_service_history', 'production_date',
       'energy_efficiency_class', 'new', 'combined_fuel_cons'],
      dtype='object')

In [18]:
for col in df_notnull.columns:
    print(first_looking(col))
    print()

COLUMN NAME    :  make_model
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  212
--------------------------------
Ford Focus                  99
Opel Insignia               97
Opel Astra                  94
Opel Corsa                  89
Ford Fiesta                 80
Volvo V60                   70
Opel Grandland X            62
Volvo XC60                  54
Ford Kuga                   53
Toyota Yaris                48
Opel Crossland X            48
SEAT Leon                   46
Volvo XC40                  44
Volvo V90                   38
Skoda Fabia                 34
Nissan Qashqai              34
Opel Cascada                34
Ford Mustang                34
Toyota Corolla              32
Toyota Aygo X               31
Renault Clio                31
Toyota C-HR                 30
Volvo V90 Cross Country     30
Volvo XC90                  28
Hyundai i30                 28
Peugeot 308                 26
Mercedes-Benz C 200         26
Me

In [19]:
first_looking("make_model", "make", "model")

COLUMN NAME    :  make_model
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  212
--------------------------------
Ford Focus                  99
Opel Insignia               97
Opel Astra                  94
Opel Corsa                  89
Ford Fiesta                 80
Volvo V60                   70
Opel Grandland X            62
Volvo XC60                  54
Ford Kuga                   53
Toyota Yaris                48
Opel Crossland X            48
SEAT Leon                   46
Volvo XC40                  44
Volvo V90                   38
Skoda Fabia                 34
Nissan Qashqai              34
Opel Cascada                34
Ford Mustang                34
Toyota Corolla              32
Toyota Aygo X               31
Renault Clio                31
Toyota C-HR                 30
Volvo V90 Cross Country     30
Volvo XC90                  28
Hyundai i30                 28
Peugeot 308                 26
Mercedes-Benz C 200         26
Me

In [20]:
df[['make_model', 'make', 'model', "short_description"]].sample(20)

,make_model,make,model,short_description
1985,Ford Fiesta,Ford,Fiesta,1.0 EcoBoost S&S ST-LINE
966,Peugeot 2008,Peugeot,2008,1.2 PureTech Allure Klima ParkAssist Kamera
711,Opel Astra,Opel,Astra,K Sports Tourer 1.5 D Edition LED 2xKlima
1160,SEAT Leon,SEAT,Leon,FR Plus 1.5 eTSI 150 PS DSG
27,Mercedes-Benz C 63 AMG,Mercedes-Benz,C 63 AMG,Cabrio*Comand*Perf.-Abgas*Night*Driver's
918,Renault Megane,Renault,Megane,Limited 1.5 Blue DCI 115
59,Mercedes-Benz C 200,Mercedes-Benz,C 200,Cabrio AMG Line Leder + Verdeck Rot/360°/ Autom.
347,Mercedes-Benz S 580,Mercedes-Benz,S 580,e L 9G*AMG*Chauffeur-Paket*DigitalLight*Pano*HUD*
201,Mercedes-Benz GLS 400,Mercedes-Benz,GLS 400,d 4MATIC AMG Line/Navi/SHD/Distronic/LED Styling
1693,Ford EcoSport,Ford,EcoSport,Active 1.0 *LED*B&O*BLIS*SHZ*CAM*PDC*


In [21]:
df['model'].str.contains(r'^\s*$', na=False).value_counts()

False    2351
True       13
Name: model, dtype: int64

In [22]:
df["model"] = df["model"].replace(r'^\s*$', np.nan, regex=True)
df[df["model"].isna()][['make_model', 'make', 'model']]

,make_model,make,model
5,Mercedes-Benz,Mercedes-Benz,NaN
15,Mercedes-Benz,Mercedes-Benz,NaN
53,Mercedes-Benz,Mercedes-Benz,NaN
54,Mercedes-Benz,Mercedes-Benz,NaN
76,Mercedes-Benz,Mercedes-Benz,NaN
302,Mercedes-Benz,Mercedes-Benz,NaN
308,Mercedes-Benz,Mercedes-Benz,NaN
321,Mercedes-Benz,Mercedes-Benz,NaN
900,Renault,Renault,NaN
902,Renault,Renault,NaN


In [23]:
df.iloc[162]

make_model                                               Mercedes-Benz E 300
short_description           E300d Cp. AMG+DISTR+PANO+BURM MEM+AMBIE+ILS+360°
make                                                           Mercedes-Benz
model                                                                 E 300 
location                            Hans-Schardt-Straße 2,  66822 Lebach, DE
price                                                               50988.00
body_type                                                              Coupe
type                                                                    Used
doors                                                                   2.00
warranty                                                                 24 
mileage                                                             57800.00
first_registration                                                   11/2019
gearbox                                                            Automatic

In [24]:
# Let's define models and extract them from "short_description" feature
model_pattern = '(Mitecki)|(BJ40)|(Business)|(1300)|(TURBO)|(bj 42)|(4cv)|(B 220)|(E 63)|(S 580e)|(SL 43)|(EDITION 1)|(A 160)|(GLE 63)|(C 63)|(C 300)|(C 180)|(A 200)|(C 200)|(A 180)|(A 180)|(Clase GLK)|(Clase SL)|(ANDERE)|(220)|(E 500)|(GLA 250)|(4Matic)|(C 43)|(200CDI)|(AMG GT)|(AMG Autom)|(AMG-Night)|(4M)|(2.0 CDTi)|(1.4 Turbo)|(1.4 S/S)|(1.4 T)|(Cabrio 1.4)|(Cabrio 1.6)|(Cabrio 2.0)|(Olympia)|(Astra)|(Adam)|(i30)|(Twizy)|(Alpine)|(Zen)|(Spider)|(Intens)|(COUPE)|(Cabriolet)|(124 Spider)|(Qubo)|(Ultimate AWD)|(1st Edition AWD)|(240 Z)|(Buckel)|(CRDI)|(RS200)|(SPYDER)|(GR Supra)|(2,0 T-Bar)|(1.8)|(Mach E)|(370 Z)|(Zagato)|(Tornado)|(Essential)|(1100D)|(Topolino Lotus)|(850)|(Neuer 500)|(124 Spyder)|(R.S. Line)|(Blue dCi)|(1500)|(Coupè)|(201)|(Berline|(Berlina)|(403)|(600)|(850)|(124)|(128 sport)|(Cupra)|(COMFORT TCe)|(Abarth)|(Coupé)|(X-MODE)|(Family Active)|(LJ70)|(Auris Touring Sports)|(RAV)|(Cabrio)|(T-Bird)|(MUSTANG)|(RAPTOR)|(GALAXIE SUNLINER)|(HOT ROD)|(Spaceback)|(Datsun)|(Skyline)|(Roadster)|(1.0 T-GDI)|(Mild-Hybrid)|(BAYON)|(I30N)|(P 1800 E)|(P1800E)|(P1800)|(2.4 Momentum)|(PV544 Sport)|(444))'

# Let's create a new feature named "extracted_models" using model_pattern
df["extracted_models"] = (df["short_description"].str.extract(model_pattern, flags=re.IGNORECASE)).bfill(axis=1)[0]

# let's explore which "make_model", "make", and "model" have NaN values
df[df["model"].isna()][['make_model', 'make', 'model', "extracted_models", "short_description"]]

,make_model,make,model,extracted_models,short_description
5,Mercedes-Benz,Mercedes-Benz,NaN,SL 43,SL 43 DIGITAL LIGHT+BURMESTER+DISTRO+HUD+MEMORY
15,Mercedes-Benz,Mercedes-Benz,NaN,ANDERE,ANDERE 43 Premium-Navi*360°Ka.*AIRSCARF
53,Mercedes-Benz,Mercedes-Benz,NaN,C 180,C 180 Cabrio 9G-TRONIC AMG Line
54,Mercedes-Benz,Mercedes-Benz,NaN,C 200,C 200 Cabrio 9G-TRONIC AMG Line
76,Mercedes-Benz,Mercedes-Benz,NaN,AMG GT,AMG GT S E AMG PERFORMANCE grau magno HUD+CARBON
302,Mercedes-Benz,Mercedes-Benz,NaN,C 300,C 300 e T 9G AMG*Pano*Distronic*Digital-Light*...
308,Mercedes-Benz,Mercedes-Benz,NaN,E 63,E 63 AMG E 63 S AMG 4M+ MBUXHighE+PD+Burm+Mbea...
321,Mercedes-Benz,Mercedes-Benz,NaN,S 580e,S 580e 4M L 9G*AMG*Chauffeur*Pano*Digital-Light*
900,Renault,Renault,NaN,Zen,Zen 1.3 TCe LED+Navi+Keyless+Rückfahrkam.+PDC...
902,Renault,Renault,NaN,R.S. Line,R.S. Line TCe 140 *Navi *360°*KLIMA


In [25]:
df["model"].isna().sum()

13

In [26]:
df['model'].fillna(df['extracted_models']).isnull().sum()


0

In [27]:
df[['make_model', 'make', 'model', "extracted_models", "short_description"]]

,make_model,make,model,extracted_models,short_description
0,Mercedes-Benz E 400,Mercedes-Benz,E 400,4M,d 4M AMG/Com/ILS/Distr/KeyG/360/Standh/ABC Autom.
1,Mercedes-Benz AMG GT,Mercedes-Benz,AMG GT,NaN,Mercedes- Keramik/Carbon/Burmester/Night Distr...
2,Mercedes-Benz C 400,Mercedes-Benz,C 400,NaN,AMG/Comand/Distr/KeyGO/Sitzklima/Memory Distronic
3,Mercedes-Benz AMG GT,Mercedes-Benz,AMG GT,Roadster,Roadster COMAND/Burmester/Memory/Kamera Autom.
4,Mercedes-Benz AMG GT,Mercedes-Benz,AMG GT,Roadster,GT S Roadster KAMERA+BURMESTER+MEMORY+AIRSCARF
...,...,...,...,...,...
2359,Volvo S60,Volvo,S60,NaN,R Design B5 Benzin EU6d LED Navi StandHZG Keyless
2360,Volvo S90,Volvo,S90,NaN,R Design AWD B5 Diesel EU6d Leder LED Navi Sta...
2361,Volvo S90,Volvo,S90,NaN,D4 Geartronic R Design
2362,Volvo S60,Volvo,S60,NaN,R Design B4 Benzin EU6d LED Navi StandHZG Keyless


In [28]:
df[(df["model"].isna()) & (df["extracted_models"].isna())][['make_model', 'make', 'model', "extracted_models", "short_description"]]

,make_model,make,model,extracted_models,short_description


In [29]:
model_index_nan = df[df["model"].isna()][['make_model', 'make', 'model', "extracted_models", "short_description"]].index

model_index_nan

Int64Index([5, 15, 53, 54, 76, 302, 308, 321, 900, 902, 1399, 1468, 2026], dtype='int64')

In [30]:
df['model'].fillna(df['extracted_models'], inplace=True)

display(df['model'].isnull().sum())
display(df['make_model'].isnull().sum())

0

0

In [31]:
df[df["model"].isna()][['make_model', 'make', 'model', "extracted_models", "short_description"]]

,make_model,make,model,extracted_models,short_description


In [32]:
display(df.shape)  # Gives the shape of df BEFORE dropping NaN values in 'model'

df.dropna(subset=['model'], inplace=True)

display(df['model'].isnull().sum())  # Gives the total number of NaN values in 'model' AFTER dropping them 

display(df.shape)  # Gives the shape of df AFTER dropping NaN values in 'model'

(2364, 34)

0

(2364, 34)

In [33]:
df["modified_make_model"] = df["make"] + " " + df["model"]
modified_make_model = df.pop("modified_make_model")
df.insert(1, 'modified_make_model', modified_make_model)
df["modified_make_model"]

0        Mercedes-Benz E 400 
1       Mercedes-Benz AMG GT 
2        Mercedes-Benz C 400 
3       Mercedes-Benz AMG GT 
4       Mercedes-Benz AMG GT 
                ...          
2359               Volvo S60 
2360               Volvo S90 
2361               Volvo S90 
2362               Volvo S60 
2363               Volvo S60 
Name: modified_make_model, Length: 2364, dtype: object

In [34]:
df[["make_model", "modified_make_model"]]

,make_model,modified_make_model
0,Mercedes-Benz E 400,Mercedes-Benz E 400
1,Mercedes-Benz AMG GT,Mercedes-Benz AMG GT
2,Mercedes-Benz C 400,Mercedes-Benz C 400
3,Mercedes-Benz AMG GT,Mercedes-Benz AMG GT
4,Mercedes-Benz AMG GT,Mercedes-Benz AMG GT
...,...,...
2359,Volvo S60,Volvo S60
2360,Volvo S90,Volvo S90
2361,Volvo S90,Volvo S90
2362,Volvo S60,Volvo S60


In [35]:
df['modified_make_model'].fillna(df['make_model'], inplace=True)

display(df['model'].isnull().sum())
display(df['make_model'].isnull().sum())
display(df['modified_make_model'].isnull().sum())

0

0

0

In [36]:
df['make_model'] = df['modified_make_model']
df.iloc[162]

make_model                                              Mercedes-Benz E 300 
modified_make_model                                     Mercedes-Benz E 300 
short_description           E300d Cp. AMG+DISTR+PANO+BURM MEM+AMBIE+ILS+360°
make                                                           Mercedes-Benz
model                                                                 E 300 
location                            Hans-Schardt-Straße 2,  66822 Lebach, DE
price                                                               50988.00
body_type                                                              Coupe
type                                                                    Used
doors                                                                   2.00
warranty                                                                 24 
mileage                                                             57800.00
first_registration                                                   11/2019

In [37]:
df["make_model"].replace(["Ford SPYDER", "Ford MUSTANG", "Renault Grand Espace", "Fiat Spider", "Fiat Spider Europa", "Fiat 1300", "Opel Zafira Tourer", "Opel Zafira Life", "Toyota GR Supra", "Ford Grand C-Max", "Volvo P 1800 E", "Volvo P1800E", "Volvo coupe", "Toyota Auris Touring Sports", "Ford Mustang Mach-E\t", "Toyota Prius+", "Toyota 1.8", "Mercedes-Benz AMG-Night", "Mercedes-Benz Clase GLK", "Mercedes-Benz Clase SL", "Ford Hot Rod", "Ford HOT ROD", "Ford Hot rod", "Fiat 124 Coupè", "Fiat COUPÈ", "Fiat Coupè", "Toyota Corolla Verso", "Toyota Verso-S", "Renault Alpine", "Hyundai KONA Elektro", "Hyundai 1.0 T-GDi", "Hyundai H 350", "Hyundai H-1", "Hyundai ACCENT", "Hyundai CRDI", "Renault"],
                         ["Ford Mustang", "Ford Mustang", "Renault Espace", "Fiat 124 Spider", "Fiat 124 Spider", "Fiat 130", "Opel Zafira", "Opel Zafira", "Toyota Supra", "Ford C-Max", "Volvo P1800", "Volvo P1800", "Volvo Coupe", "Toyota Auris", "Ford Mustang Mach-E", "Toyota Prius", "Toyota MR 2", "Mercedes-Benz AMG GT", "Mercedes-Benz Clase", "Mercedes-Benz Clase", "Ford Hot Rod", "Ford Hot Rod", "Ford Hot Rod", "Fiat 124 Coupe", "Fiat 124 Coupe", "Fiat 124 Coupe", "Toyota Verso", "Toyota Verso", "Renault Alpine A110", "Hyundai KONA", "Hyundai 1.0 T-GDI", "Hyundai H", "Hyundai H", "Hyundai Accent", "Hyundai Accent", "Renault Megane"], inplace=True)

In [38]:
df[["make_model", "make", "model"]] = df[["make_model", "make", "model"]].apply(lambda x: x.str.title())

In [39]:
df.drop(["modified_make_model", "short_description", "extracted_models"], axis=1, inplace=True)
df.head()

,make_model,make,model,location,price,body_type,type,doors,warranty,mileage,first_registration,gearbox,fuel_type,seller,seats,engine_size,gears,co_emissions,drivetrain,cylinders,comfort_&_convenience,entertainment_&_media,safety_&_security,extras,empty_weight,full_service_history,upholstery,production_date,previous_owner,energy_efficiency_class,new,combined_fuel_cons
0,Mercedes-Benz E 400,Mercedes-Benz,E 400,"Julius-Rütgers-Str. 1, 15537 Erkner, DE",64870.00,Convertible,Used,2.00,24,78473.00,01/2020,Automatic,Diesel (Particle filter),Dealer,4.00,2925.00,9.00,162.00,4WD,6.00,"['360° camera, Air conditioning, Air suspensio...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Ambient lighting, Automatically ...",2010.00,Yes,NaN,2020.00,1.00,efficient,Mercedes-Benz E 400,6.20
1,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"Wormser Landstraße 194, 67346 Speyer, DE",197500.00,Convertible,Employee's car,2.00,0,9550.00,04/2021,Automatic,Super 95,Dealer,2.00,3982.00,7.00,298.00,Rear,8.00,"['Air conditioning, Armrest, Automatic climate...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Alarm system, ...","Alloy wheels, Ambient lighting, Automatically ...",1735.00,Yes,NaN,2021.00,1.00,unefficient,Mercedes-Benz AMG GT,13.00
2,Mercedes-Benz C 400,Mercedes-Benz,C 400,"Autofocus 3, 15517 Fürstenwalde, DE",63370.00,Convertible,Employee's car,2.00,24,20145.00,01/2021,Automatic,Super 95,Dealer,4.00,2996.00,9.00,196.00,4WD,6.00,"['Armrest, Automatic climate control, 2 zones,...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Ambient lighting, Automatically ...",1825.00,Yes,NaN,2021.00,1.00,unefficient,Mercedes-Benz C 400,8.60
3,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"Meckenheimer Straße 12, 53359 Rheinbach, DE",175000.00,Convertible,Used,2.00,24,3561.00,07/2021,Automatic,Super 95,Dealer,2.00,3982.00,7.00,296.00,NaN,8.00,"['Air conditioning, Armrest, Automatic climate...","['Android Auto, Apple CarPlay, Digital radio, ...","['ABS, Blind spot monitor, Central door lock, ...","Alloy wheels, Automatically dimming interior m...",1700.00,No,NaN,2021.00,1.00,unefficient,Mercedes-Benz AMG GT,12.90
4,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"St.-Georgener-Straße 1, 79111 Freiburg, DE",149890.00,Convertible,Used,3.00,Yes,23851.00,12/2019,Automatic,Super E10 95 (Particle filter),Dealer,2.00,3982.00,7.00,262.00,NaN,8.00,"['Air conditioning, Armrest, Automatic climate...","['Apple CarPlay, Bluetooth, Digital radio, Han...","['ABS, Adaptive Cruise Control, Alarm system, ...","Alloy wheels (20""), Ambient lighting, Automati...",1700.00,Yes,Full leather,2019.00,2.00,unefficient,Mercedes-Benz AMG GT,11.50


In [40]:
first_looking("make_model", "make", "model")

COLUMN NAME    :  make_model
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  221
--------------------------------
Ford Focus                   99
Opel Insignia                97
Opel Astra                   94
Opel Corsa                   89
Ford Fiesta                  80
Volvo V60                    70
Opel Grandland X             62
Volvo Xc60                   54
Ford Kuga                    53
Opel Crossland X             48
Toyota Yaris                 48
Seat Leon                    46
Volvo Xc40                   44
Volvo V90                    38
Skoda Fabia                  34
Ford Mustang                 34
Nissan Qashqai               34
Opel Cascada                 34
Toyota Corolla               32
Renault Clio                 31
Toyota Aygo X                31
Volvo V90 Cross Country      30
Toyota C                     30
Hyundai I30                  28
Volvo Xc90                   28
Peugeot 308                  26
Mercede

In [41]:
first_looking("body_type")

COLUMN NAME    :  body_type
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  6
--------------------------------
Off-Road/Pick-up    684
Station wagon       609
Sedan               461
Compact             293
Coupe               175
Convertible         142
Name: body_type, dtype: int64




In [42]:
df['body_type'] = df['body_type'].str.title()

In [43]:
first_looking("body_type")

COLUMN NAME    :  body_type
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  6
--------------------------------
Off-Road/Pick-Up    684
Station Wagon       609
Sedan               461
Compact             293
Coupe               175
Convertible         142
Name: body_type, dtype: int64




FILLING MISSING VALUES

In [44]:
df_isnull = df[df.columns[df.isnull().any()]]
df_isnull.columns

Index(['fuel_type', 'seats', 'engine_size', 'gears', 'drivetrain', 'cylinders',
       'comfort_&_convenience', 'entertainment_&_media', 'safety_&_security',
       'extras', 'empty_weight', 'upholstery', 'previous_owner'],
      dtype='object')

In [45]:
interact(show_missing_values, limit=(0, 100, 5))

interactive(children=(IntSlider(value=50, description='limit', step=5), Output()), _dom_classes=('widget-inter…

<function __main__.show_missing_values(limit)>

In [46]:
first_looking("engine_size")

COLUMN NAME    :  engine_size
--------------------------------
per_of_nulls   :  % 0.25
num_of_nulls   :  6
num_of_uniques :  102
--------------------------------
1969.00    267
999.00     191
1199.00    185
1598.00    169
998.00     157
1498.00     82
1991.00     67
3982.00     67
1332.00     56
1998.00     54
1997.00     53
1984.00     50
1995.00     49
1798.00     43
1499.00     42
2996.00     40
1956.00     40
1364.00     39
1968.00     38
1987.00     38
2925.00     37
1490.00     37
2999.00     34
1496.00     32
1197.00     31
1497.00     29
5038.00     26
1950.00     25
898.00      24
1398.00     23
1477.00     22
1229.00     17
1461.00     16
1399.00     13
1618.00     12
1482.00     11
2261.00     11
2488.00     10
1368.00      9
1242.00      9
2143.00      9
1597.00      9
1993.00      8
1595.00      8
1999.00      8
2998.00      8
1084.00      8
1591.00      8
875.00       7
4951.00      6
1560.00      6
2792.00      6
1796.00      6
3696.00      6
NaN          6
1395.00     

In [47]:
df["engine_size"].fillna("-", inplace=True)

In [48]:
df.groupby(["make_model", "body_type","engine_size"]).price.describe()

count     mean      std      min  \
make_model    body_type        engine_size                                     
Dacia Duster  Off-Road/Pick-Up 1197.00       1.00 14990.00      NaN 14990.00   
                               1332.00       2.00 20070.00  1527.35 18990.00   
                               1461.00       1.00 21690.00      NaN 21690.00   
                               1598.00       1.00 16490.00      NaN 16490.00   
Dacia Jogger  Station Wagon    999.00        4.00 23207.50  1364.61 21750.00   
...                                           ...      ...      ...      ...   
Volvo Xc40    Off-Road/Pick-Up 1969.00      22.00 31586.36  3684.71 25480.00   
Volvo Xc60    Compact          1969.00       2.00 72940.00     0.00 72940.00   
              Off-Road/Pick-Up 1969.00      48.00 50439.12 17273.21 27990.00   
                               1984.00       4.00 17935.00  1091.19 16990.00   
Volvo Xc90    Off-Road/Pick-Up 1969.00      28.00 64244.57 22843.15 29980.00   

                                                25%      50%      75%      max  
make_model    body_type        engine_size                                      
Dacia Duster  Off-Road/Pick-Up 1197.00     14990.00 14990.00 14990.00 14990.00  
                               1332.00     19530.00 20070.00 20610.00 21150.00  
                               1461.00     21690.00 21690.00 21690.00 21690.00  
                               1598.00     16490.00 16490.00 16490.00 16490.00  
Dacia Jogger  Station Wagon    999.00      22237.50 23195.00 24165.00 24690.00  
...                                             ...      ...      ...      ...  
Volvo Xc40    Off-Road/Pick-Up 1969.00     29440.00 31770.00 34862.50 36490.00  
Volvo Xc60    Compact          1969.00     72940.00 72940.00 72940.00 72940.00  
              Off-Road/Pick-Up 1969.00     36074.75 46930.00 68915.00 83890.00  
                               1984.00     16990.00 17935.00 18880.00 18880.00  
Volvo Xc90    Off-Road/Pick-Up 1969.00     44970.00 64220.00 84780.00 97450.00  

[635 rows x 8 columns]

## The engine_size of cars changes by their make_models & body_type most of the time. So it is more beneficial to decide to fill missing values with mode value of corresponding group.

In [49]:
df["engine_size"].replace("-", np.nan, inplace=True)

In [50]:
def fill(df, group_col1, group_col2, col_name, method): # method can be either "mode" or "mean" or "median" or "ffill"
    
    '''Fills the missing values with "mode/mean/median/ffill/bfill method" according to double-stage grouping'''
    
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])

    elif method == "mean":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df[col_name].mean(), inplace = True)
        
    elif method == "median":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df[col_name].median(), inplace = True)
        
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("COLUMN NAME    : ", col_name)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col_name].isnull().sum()*100 / df.shape[0], 2))
    print("num_of_nulls   : ", df[col_name].isnull().sum())
    print("num_of_uniques : ", df[col_name].nunique())
    print("--------------------------------")
    print(df[col_name].value_counts(dropna = False).sort_index())

In [51]:
fill(df, "make_model", "body_type", "engine_size", "mode")

COLUMN NAME    :  engine_size
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  102
--------------------------------
0.00         4
875.00       7
898.00      24
998.00     158
999.00     192
1084.00      8
1149.00      3
1197.00     31
1198.00      5
1199.00    186
1200.00      1
1229.00     17
1242.00      9
1248.00      4
1329.00      4
1332.00     56
1333.00      3
1341.00      5
1342.00      1
1353.00      1
1364.00     39
1368.00      9
1388.00      1
1390.00      2
1395.00      6
1398.00     23
1399.00     13
1400.00      1
1422.00      1
1461.00     16
1477.00     22
1482.00     11
1490.00     37
1495.00      1
1496.00     32
1497.00     29
1498.00     82
1499.00     42
1500.00      1
1560.00      6
1580.00      2
1582.00      2
1591.00      8
1595.00      8
1596.00      3
1597.00      9
1598.00    169
1618.00     12
1685.00      2
1686.00      2
1749.00      1
1796.00      6
1798.00     44
1950.00     25
1956.00     40
1968.00     3

In [52]:
first_looking("doors")

COLUMN NAME    :  doors
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  4
--------------------------------
5.00    1889
2.00     216
3.00     132
4.00     127
Name: doors, dtype: int64




In [53]:
# The number of doors of cars mostly changes by their make_models and body_types. So it's decided to fill missing values with mode value of corresponding group.
# This is the nested for loop where it's used in fill_mode(df, group_col1, group_col2, col_name) DEF to be generalized for all data

for group1 in list(df["make_model"].unique()):
    for group2 in list(df["body_type"].unique()):
        cond1 = df["make_model"]==group1
        cond2 = (df["make_model"]==group1) & (df["body_type"]==group2)
        mode1 = list(df[cond1]["doors"].mode())
        mode2 = list(df[cond2]["doors"].mode())
        if mode2 != []:
            df.loc[cond2, "doors"] = df.loc[cond2, "doors"].fillna(df[cond2]["doors"].mode()[0])
        elif mode1 != []:
            df.loc[cond2, "doors"] = df.loc[cond2, "doors"].fillna(df[cond1]["doors"].mode()[0])
        else:
            df.loc[cond2, "doors"] = df.loc[cond2, "doors"].fillna(df["doors"].mode()[0])

In [54]:
# df["doors"].value_counts(dropna=False)
first_looking("doors")

COLUMN NAME    :  doors
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  4
--------------------------------
5.00    1889
2.00     216
3.00     132
4.00     127
Name: doors, dtype: int64




In [55]:
def fill_mode(df, group_col1, group_col2, col_name):
    
    for group1 in list(df[group_col1].unique()):
        for group2 in list(df[group_col2].unique()):
            cond1 = df[group_col1]==group1
            cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
            mode1 = list(df[cond1][col_name].mode())
            mode2 = list(df[cond2][col_name].mode())
            if mode2 != []:
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
            elif mode1 != []:
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
            else:
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])
    
    print("column name    : ", col_name)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col_name].isnull().sum()*100 / df.shape[0], 2))
    print("num_of_nulls   : ", df[col_name].isnull().sum())
    print("num_of_uniques : ", df[col_name].nunique())
    print("--------------------------------")
    print(df[col_name].value_counts(dropna = False))

In [56]:
# In fact there is no need to run this DEF since all NaNs manually were filled above by for nested loop
fill_mode(df, "make_model", "body_type", "doors")

column name    :  doors
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  4
--------------------------------
5.00    1889
2.00     216
3.00     132
4.00     127
Name: doors, dtype: int64


In [57]:
first_looking("gears")

COLUMN NAME    :  gears
--------------------------------
per_of_nulls   :  % 10.49
num_of_nulls   :  248
num_of_uniques :  8
--------------------------------
6.00     915
8.00     458
5.00     253
NaN      248
7.00     197
9.00     192
1.00      85
10.00     14
4.00       2
Name: gears, dtype: int64




In [58]:
df[df["fuel_type"]=="Electric"].gears.value_counts(dropna=False)

Series([], Name: gears, dtype: int64)

In [59]:
df["gears"].fillna("-", inplace=True)

In [60]:
df.groupby(["make_model", "body_type", "gearbox", "gears"]).price.describe()

count     mean      std  \
make_model     body_type        gearbox   gears                            
Dacia Duster   Off-Road/Pick-Up Manual    5.00    1.00 16490.00      NaN   
                                          6.00    4.00 19205.00  3042.52   
Dacia Jogger   Station Wagon    Manual    6.00    4.00 23207.50  1364.61   
Dacia Logan    Station Wagon    Manual    5.00    2.00 10990.00     0.00   
Dacia Sandero  Compact          Automatic 1.00    1.00 18990.00      NaN   
...                                                ...      ...      ...   
Volvo Xc40     Off-Road/Pick-Up Manual    6.00   12.00 29681.67  2205.12   
Volvo Xc60     Compact          Automatic 8.00    2.00 72940.00     0.00   
               Off-Road/Pick-Up Automatic 8.00   44.00 52210.86 16933.96   
                                Manual    6.00    8.00 24442.50  7342.62   
Volvo Xc90     Off-Road/Pick-Up Automatic 8.00   28.00 64244.57 22843.15   

                                                     min      25%      50%  \
make_model     body_type        gearbox   gears                              
Dacia Duster   Off-Road/Pick-Up Manual    5.00  16490.00 16490.00 16490.00   
                                          6.00  14990.00 17990.00 20070.00   
Dacia Jogger   Station Wagon    Manual    6.00  21750.00 22237.50 23195.00   
Dacia Logan    Station Wagon    Manual    5.00  10990.00 10990.00 10990.00   
Dacia Sandero  Compact          Automatic 1.00  18990.00 18990.00 18990.00   
...                                                  ...      ...      ...   
Volvo Xc40     Off-Road/Pick-Up Manual    6.00  25480.00 28880.00 30385.00   
Volvo Xc60     Compact          Automatic 8.00  72940.00 72940.00 72940.00   
               Off-Road/Pick-Up Automatic 8.00  28680.00 37890.00 48460.00   
                                Manual    6.00  16990.00 18407.50 23435.00   
Volvo Xc90     Off-Road/Pick-Up Automatic 8.00  29980.00 44970.00 64220.00   

                                                     75%      max  
make_model     body_type        gearbox   gears                    
Dacia Duster   Off-Road/Pick-Up Manual    5.00  16490.00 16490.00  
                                          6.00  21285.00 21690.00  
Dacia Jogger   Station Wagon    Manual    6.00  24165.00 24690.00  
Dacia Logan    Station Wagon    Manual    5.00  10990.00 10990.00  
Dacia Sandero  Compact          Automatic 1.00  18990.00 18990.00  
...                                                  ...      ...  
Volvo Xc40     Off-Road/Pick-Up Manual    6.00  30980.00 31980.00  
Volvo Xc60     Compact          Automatic 8.00  72940.00 72940.00  
               Off-Road/Pick-Up Automatic 8.00  68990.00 83890.00  
                                Manual    6.00  29470.00 33910.00  
Volvo Xc90     Off-Road/Pick-Up Automatic 8.00  84780.00 97450.00  

[616 rows x 8 columns]

The number of gears of cars changes by their make_models, body_types, and gearbox most of the time. So it's decided to fill missing values with mode value of corresponding group.

In [61]:
df["gears"].replace("-", np.nan, inplace=True)

In [62]:
for group1 in df["make_model"].unique():
    for group2 in df["body_type"].unique():
        for group3 in df["gearbox"].unique():
            cond1 = df["make_model"] == group1
            cond2 = (df["make_model"] == group1) & (df["body_type"] == group2)
            cond3 = (df["make_model"] == group1) & (df["body_type"] == group2) & (df["gearbox"] == group3)
            mode1 = list(df[cond1]["gears"].mode())
            mode2 = list(df[cond2]["gears"].mode())
            mode3 = list(df[cond3]["gears"].mode())
            if mode3 != []:
                df.loc[cond3, "gears"] = df.loc[cond3, "gears"].fillna(df[cond3]["gears"].mode()[0])
            elif mode2 != []:
                df.loc[cond3, "gears"] = df.loc[cond3, "gears"].fillna(df[cond2]["gears"].mode()[0])
            elif mode1 != []:
                df.loc[cond3, "gears"] = df.loc[cond3, "gears"].fillna(df[cond1]["gears"].mode()[0])
            else:
                df.loc[cond3, "gears"] = df.loc[cond3, "gears"].fillna(df["gears"].mode()[0])

In [63]:
first_looking("gears")

COLUMN NAME    :  gears
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  8
--------------------------------
6.00     975
8.00     467
5.00     283
9.00     273
7.00     214
1.00     131
10.00     19
4.00       2
Name: gears, dtype: int64




In [64]:
df[df["gears"].isna()][["make_model", "body_type", "gearbox"]]

,make_model,body_type,gearbox


In [65]:
first_looking("gearbox")

COLUMN NAME    :  gearbox
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  2
--------------------------------
Automatic    1287
Manual       1077
Name: gearbox, dtype: int64




In [66]:
df[df["fuel_type"]=="Electric"].gearbox.value_counts(dropna=False)

Series([], Name: gearbox, dtype: int64)

In [67]:
df["gearbox"].fillna("-", inplace=True)

In [68]:
df.groupby(["make_model", "body_type", "gearbox"]).price.describe()

count      mean  \
make_model                body_type        gearbox                      
Dacia Duster              Off-Road/Pick-Up Manual      5.00  18662.00   
Dacia Jogger              Station Wagon    Manual      4.00  23207.50   
Dacia Logan               Station Wagon    Manual      2.00  10990.00   
Dacia Sandero             Compact          Automatic   1.00  18990.00   
                                           Manual      3.00  17690.00   
                          Off-Road/Pick-Up Manual      1.00  10950.00   
                          Sedan            Manual      4.00  12247.50   
Fiat 124 Spider           Convertible      Manual      2.00  23470.00   
Fiat 500                  Compact          Automatic   1.00  14475.00   
                                           Manual      4.00  17856.00   
                          Off-Road/Pick-Up Automatic   1.00  29985.00   
                          Sedan            Manual      5.00  12285.80   
Fiat 500C                 Convertible      Automatic   1.00  19990.00   
                                           Manual     11.00  16182.82   
Fiat 500X                 Off-Road/Pick-Up Automatic   1.00  16990.00   
                                           Manual      1.00  15350.00   
                          Sedan            Manual      1.00  23990.00   
Fiat Punto                Compact          Manual      1.00   5799.00   
Fiat Tipo                 Sedan            Manual      4.00  17427.25   
                          Station Wagon    Automatic   1.00  31285.00   
                                           Manual      2.00  11730.00   
Ford Bronco               Off-Road/Pick-Up Automatic   1.00  47950.00   
Ford Ecosport             Compact          Manual      4.00  23865.00   
Ford Edge                 Off-Road/Pick-Up Automatic   4.00  34775.00   
Ford Explorer             Off-Road/Pick-Up Automatic   1.00  67990.00   
Ford Fiesta               Compact          Automatic   4.00  21255.00   
                                           Manual     32.00  15803.03   
                          Coupe            Manual      1.00  20740.00   
                          Sedan            Automatic   9.00  21363.33   
                                           Manual     34.00  15609.74   
Ford Focus                Coupe            Manual      2.00  26920.00   
                          Sedan            Automatic   6.00  28580.00   
                                           Manual     31.00  24826.26   
                          Station Wagon    Automatic  15.00  25704.27   
                                           Manual     45.00  21962.84   
Ford Grand C              Station Wagon    Manual      1.00  19290.00   
Ford Grand Tourneo        Station Wagon    Manual      1.00  25180.00   
Ford Kuga                 Off-Road/Pick-Up Automatic  15.00  34638.00   
                                           Manual     38.00  26618.37   
Ford Mondeo               Sedan            Automatic   1.00  33980.00   
                                           Manual      2.00  16990.00   
                          Station Wagon    Automatic  16.00  27370.62   
                                           Manual      3.00  17959.67   
Ford Mustang              Convertible      Automatic   6.00  63266.67   
                                           Manual      4.00  40939.50   
                          Coupe            Automatic  15.00  55668.53   
                                           Manual      9.00  60720.00   
Ford Puma                 Compact          Manual      1.00  24550.00   
                          Off-Road/Pick-Up Automatic   1.00  26950.00   
                                           Manual      7.00  26007.71   
Ford Ranger               Off-Road/Pick-Up Automatic   3.00  40636.67   
                                           Manual      1.00  25890.00   
Ford S                    Sedan            Manual      1.00  32690.00   
Ford Tourneo Courier      Sedan        

The type of gearbox of cars changes by their make_models and body_types most of the time. So it's decided to fill missing values with mode value of corresponding group.

In [69]:

df["gearbox"] = df["gearbox"].replace('-', np.nan)

In [70]:
fill(df, "make_model", "body_type", "gearbox", "mode")

COLUMN NAME    :  gearbox
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  2
--------------------------------
Automatic    1287
Manual       1077
Name: gearbox, dtype: int64


In [71]:
# This is the nested for loop where it's used in fill(df, group_col1, group_col2, col_name, method) DEF to be generalized for all data

for group1 in df["make_model"].unique():
    for group2 in df["body_type"].unique():
        for group3 in df["gearbox"].unique():
            cond1 = df["make_model"] == group1
            cond2 = (df["make_model"] == group1) & (df["body_type"] == group2)
            cond3 = (df["make_model"] == group1) & (df["body_type"] == group2) & (df["gearbox"] == group3)
            mode1 = list(df[cond1]["gears"].mode())
            mode2 = list(df[cond2]["gears"].mode())
            mode3 = list(df[cond3]["gears"].mode())
            if mode3 != []:
                df.loc[cond3, "gears"] = df.loc[cond3, "gears"].fillna(df[cond3]["gears"].mode()[0])
            elif mode2 != []:
                df.loc[cond3, "gears"] = df.loc[cond3, "gears"].fillna(df[cond2]["gears"].mode()[0])
            elif mode1 != []:
                df.loc[cond3, "gears"] = df.loc[cond3, "gears"].fillna(df[cond1]["gears"].mode()[0])
            else:
                df.loc[cond3, "gears"] = df.loc[cond3, "gears"].fillna(df["gears"].mode()[0])

In [72]:
first_looking("gears")

COLUMN NAME    :  gears
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  8
--------------------------------
6.00     975
8.00     467
5.00     283
9.00     273
7.00     214
1.00     131
10.00     19
4.00       2
Name: gears, dtype: int64




In [73]:
first_looking("fuel_type")

COLUMN NAME    :  fuel_type
--------------------------------
per_of_nulls   :  % 6.51
num_of_nulls   :  154
num_of_uniques :  17
--------------------------------
Diesel (Particle filter)                    529
Super 95                                    476
Super E10 95                                395
Regular/Benzine 91                          190
Super E10 95 (Particle filter)              189
Regular/Benzine 91 (Particle filter)        181
NaN                                         154
Super 95 (Particle filter)                  123
Gasoline                                     32
Diesel                                       31
Super Plus 98                                30
Super Plus E10 98                            13
Regular/Benzine E10 91                       10
Super Plus 98 (Particle filter)               4
Regular/Benzine E10 91 (Particle filter)      3
Super Plus E10 98 (Particle filter)           2
Gasoline (Particle filter)                    1
Hydrogen              

In [74]:
df["fuel_type"].fillna("-", inplace=True)

If we use the same transmission in both engines, the power to the wheels would be uneven and disproportionate as per the engine RPM and driving conditions. Hence we cannot couple the same manual transmission with both the engines.

In [75]:
# check if there is any pattern among ""make_model", "body_type", "gearbox", and "fuel_type" by "price"
df.groupby(["make_model", "body_type", "gearbox", "fuel_type"]).price.describe()

count  \
make_model    body_type        gearbox   fuel_type                               
Dacia Duster  Off-Road/Pick-Up Manual    Diesel (Particle filter)         1.00   
                                         Super 95                         2.00   
                                         Super 95 (Particle filter)       1.00   
                                         Super E10 95                     1.00   
Dacia Jogger  Station Wagon    Manual    Super 95                         3.00   
...                                                                        ...   
Volvo Xc60    Off-Road/Pick-Up Automatic Super E10 95 (Particle filter)   2.00   
                               Manual    Diesel (Particle filter)         8.00   
Volvo Xc90    Off-Road/Pick-Up Automatic Diesel (Particle filter)        18.00   
                                         Regular/Benzine 91               8.00   
                                         Super 95                         2.00   

                                                                            mean  \
make_model    body_type        gearbox   fuel_type                                 
Dacia Duster  Off-Road/Pick-Up Manual    Diesel (Particle filter)       21690.00   
                                         Super 95                       15740.00   
                                         Super 95 (Particle filter)     18990.00   
                                         Super E10 95                   21150.00   
Dacia Jogger  Station Wagon    Manual    Super 95                       23693.33   
...                                                                          ...   
Volvo Xc60    Off-Road/Pick-Up Automatic Super E10 95 (Particle filter) 51890.00   
                               Manual    Diesel (Particle filter)       24442.50   
Volvo Xc90    Off-Road/Pick-Up Automatic Diesel (Particle filter)       54628.33   
                                         Regular/Benzine 91             82709.75   
                                         Super 95                       76930.00   

                                                                             std  \
make_model    body_type        gearbox   fuel_type                                 
Dacia Duster  Off-Road/Pick-Up Manual    Diesel (Particle filter)            NaN   
                                         Super 95                        1060.66   
                                         Super 95 (Particle filter)          NaN   
                                         Super E10 95                        NaN   
Dacia Jogger  Station Wagon    Manual    Super 95                        1173.47   
...                                                                          ...   
Volvo Xc60    Off-Road/Pick-Up Automatic Super E10 95 (Particle filter)     0.00   
                               Manual    Diesel (Particle filter)        7342.62   
Volvo Xc90    Off-Road/Pick-Up Automatic Diesel (Particle filter)       21152.66   
                                         Regular/Benzine 91             15894.38   
                                         Super 95                           0.00   

                                                                             min  \
make_model    body_type        gearbox   fuel_type                                 
Dacia Duster  Off-Road/Pick-Up Manual    Diesel (Particle filter)       21690.00   
                                         Super 95                       14990.00   
                                         Super 95 (Particle filter)     18990.00   
                                         Super E10 95                   21150.00   
Dacia Jogger  Station Wagon    Manual    Super 95                       22400.00   
...                                                                          ...   
Volvo Xc60    Off-Road/Pick-Up Automatic Super E10 95 (Particle filter) 51890.00   
                               Manual    Diesel (Particle filter

In [76]:
df["fuel_type"] = df["fuel_type"].replace('-', np.nan)

In [77]:
#Domain Knowledge demonstrates that the fuel type of cars might change by their make_models and gear_box most of the time. So it's decided to fill missing values with mode value of corresponding group.

# fills the missing values in the "fuel_type" with the mode value based on "make_model" and "body_type"

fill(df, "make_model", "gearbox", "fuel_type", "mode")

COLUMN NAME    :  fuel_type
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  17
--------------------------------
Diesel                                       32
Diesel (Particle filter)                    577
Gasoline                                     32
Gasoline (Particle filter)                    1
Hydrogen                                      4
Regular/Benzine 91                          215
Regular/Benzine 91 (Particle filter)        198
Regular/Benzine E10 91                       10
Regular/Benzine E10 91 (Particle filter)      3
Super 95                                    528
Super 95 (Particle filter)                  124
Super E10 95                                402
Super E10 95 (Particle filter)              189
Super Plus 98                                30
Super Plus 98 (Particle filter)               4
Super Plus E10 98                            13
Super Plus E10 98 (Particle filter)           2
Name: fuel_type, dtype: i

In [78]:
first_looking("seats")

COLUMN NAME    :  seats
--------------------------------
per_of_nulls   :  % 1.06
num_of_nulls   :  25
num_of_uniques :  8
--------------------------------
5.00    1923
4.00     300
2.00      46
7.00      43
NaN       25
8.00      12
3.00      11
9.00       3
0.00       1
Name: seats, dtype: int64




In [79]:
#Domain Knowledge demonstrates that the number of seats of cars might change by their make_models and body_types most of the time. 
#So it's decided to fill missing values with mode value of corresponding group.

# # fills the missing values in the "seats" with the mode value based on "make_model" and "body_type"

fill(df, "make_model", "body_type", "seats", "mode")

COLUMN NAME    :  seats
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  8
--------------------------------
0.00       1
2.00      46
3.00      11
4.00     316
5.00    1932
7.00      43
8.00      12
9.00       3
Name: seats, dtype: int64


In [80]:
first_looking("co_emissions")

COLUMN NAME    :  co_emissions
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  234
--------------------------------
0.00      54
114.00    50
108.00    49
93.00     42
136.00    42
124.00    39
117.00    38
119.00    37
125.00    37
139.00    37
144.00    36
99.00     35
122.00    34
107.00    33
126.00    32
130.00    32
128.00    32
111.00    31
127.00    31
112.00    30
129.00    30
138.00    29
92.00     28
109.00    28
174.00    27
120.00    27
121.00    27
116.00    26
131.00    26
148.00    26
103.00    26
147.00    25
135.00    25
171.00    23
134.00    23
105.00    22
118.00    22
110.00    21
164.00    21
115.00    20
140.00    19
150.00    18
98.00     18
133.00    18
101.00    18
158.00    17
96.00     17
95.00     17
154.00    17
143.00    16
102.00    16
113.00    16
145.00    16
156.00    16
94.00     15
64.00     15
97.00     15
137.00    15
123.00    15
153.00    15
85.00     15
104.00    15
106.00    14
149.00    14
176.0

In [81]:
# All NaN values are converted to dash/hyphen sign "-" to see them in the output of describe() and explore the hidden pattern

df["co_emissions"].fillna("-", inplace=True)

In [82]:
# let's explore if there is any pattern among ""make_model", "body_type", "fuel_type", and "co_emissions" by "price"

df.groupby(["make_model", "body_type", "fuel_type", "co_emissions"]).price.describe()

count  \
make_model    body_type        fuel_type                  co_emissions          
Dacia Duster  Off-Road/Pick-Up Diesel (Particle filter)   119.00         1.00   
                               Super 95                   138.00         1.00   
                                                          149.00         1.00   
                               Super 95 (Particle filter) 138.00         1.00   
                               Super E10 95               131.00         1.00   
...                                                                       ...   
Volvo Xc90    Off-Road/Pick-Up Diesel (Particle filter)   159.00         2.00   
                               Regular/Benzine 91         47.00          2.00   
                                                          50.00          2.00   
                                                          52.00          4.00   
                               Super 95                   47.00          2.00   

                                                                           mean  \
make_model    body_type        fuel_type                  co_emissions            
Dacia Duster  Off-Road/Pick-Up Diesel (Particle filter)   119.00       21690.00   
                               Super 95                   138.00       14990.00   
                                                          149.00       16490.00   
                               Super 95 (Particle filter) 138.00       18990.00   
                               Super E10 95               131.00       21150.00   
...                                                                         ...   
Volvo Xc90    Off-Road/Pick-Up Diesel (Particle filter)   159.00       48880.00   
                               Regular/Benzine 91         47.00        70999.00   
                                                          50.00        64990.00   
                                                          52.00        97425.00   
                               Super 95                   47.00        76930.00   

                                                                         std  \
make_model    body_type        fuel_type                  co_emissions         
Dacia Duster  Off-Road/Pick-Up Diesel (Particle filter)   119.00         NaN   
                               Super 95                   138.00         NaN   
                                                          149.00         NaN   
                               Super 95 (Particle filter) 138.00         NaN   
                               Super E10 95               131.00         NaN   
...                                                                      ...   
Volvo Xc90    Off-Road/Pick-Up Diesel (Particle filter)   159.00        0.00   
                               Regular/Benzine 91         47.00         0.00   
                                                          50.00         0.00   
                                                          52.00        28.87   
                               Super 95                   47.00         0.00   

                                                                            min  \
make_model    body_type        fuel_type                  co_emissions            
Dacia Duster  Off-Road/Pick-Up Diesel (Particle filter)   119.00       21690.00   
                               Super 95                   138.00       14990.00   
                                                          149.00       16490.00   
                               Super 95 (Particle filter) 138.00       18990.00   
                               Super E10 95               131.00       21150.00   
...                                                                         ...   
Volvo Xc90    Off-Road/Pick-Up Diesel (Particle filter)   159.00       48880.00   
                               Regular/Benzine 91         47.00        70999.00   
                                                          50.00    

It seems more sensible/rational to impute missing values in two stages:

With the Mode value of Electric feature for Electric cars.

With the Median values of related make_model, body_type and fuel_type features for the others.


In [88]:
# Let's replace dash/hyphen sign "-" with np.nan

df["co_emissions"].replace("-", np.nan, inplace=True)

In [89]:
# Let's explore CO Emission values of Electric Vehicles

df.loc[df["fuel_type"] == "Electric", "co_emissions"]

Series([], Name: co_emissions, dtype: float64)

In [90]:
# Let's see the occurances of unique CO Emission values for Electric Vehicles

df[df["fuel_type"] == "Electric"]["co_emissions"].value_counts(dropna=False)

Series([], Name: co_emissions, dtype: int64)

In [93]:
# Let's explore CO Emission values of Vehicles which run with Benzine

df[df["fuel_type"] == "Benzine"]["co_emissions"].value_counts(dropna=False)

Series([], Name: co_emissions, dtype: int64)

In [94]:
first_looking("cylinders")

COLUMN NAME    :  cylinders
--------------------------------
per_of_nulls   :  % 11.0
num_of_nulls   :  260
num_of_uniques :  7
--------------------------------
4.00     1314
3.00      559
NaN       260
6.00      126
8.00       89
5.00       10
2.00        5
12.00       1
Name: cylinders, dtype: int64




In [95]:
# All NaN values are converted to dash/hyphen sign "-" to see them in the output of describe() and explore the hidden pattern

df["cylinders"].fillna("-", inplace=True)

In [96]:
# Let's replace dash/hyphen sign "-" with np.nan

df["cylinders"].replace("-", np.nan, inplace=True)

In [97]:
# fills the missing values in the "cylinders" with the mode value based on "make_model" and "body_type"

fill(df, "make_model", "body_type", "cylinders", "mode")

COLUMN NAME    :  cylinders
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  7
--------------------------------
2.00        6
3.00      651
4.00     1454
5.00       10
6.00      137
8.00      105
12.00       1
Name: cylinders, dtype: int64


However, it is preferred to drop the cylinders feature in this analysis since there have been some other features, such as hp, engine_size, and empty_weight, having fewer missing values and giving similar insight. So it's assumed to continue without the cylinders feature in this analysis.

In [98]:
df.shape

(2364, 32)

In [99]:
df.drop("cylinders", axis=1, inplace=True)

In [100]:
df.shape

(2364, 31)

In [101]:
first_looking("drivetrain")

COLUMN NAME    :  drivetrain
--------------------------------
per_of_nulls   :  % 53.05
num_of_nulls   :  1254
num_of_uniques :  3
--------------------------------
NaN       1254
 4WD       544
 Front     508
 Rear       58
Name: drivetrain, dtype: int64




In [102]:
# All NaN values are converted to dash/hyphen sign "-" to see them in the output of describe() and explore the hidden pattern

df["drivetrain"].fillna("-", inplace=True)

In [103]:
#  explore if there is any pattern among "make_model", "body_type", and "drivetrain" by "price"

df.groupby(["make_model", "body_type", "drivetrain"]).price.describe()

count      mean  \
make_model                body_type        drivetrain                    
Dacia Duster              Off-Road/Pick-Up  4WD         1.00  21690.00   
                                            Front       3.00  16823.33   
                                           -            1.00  21150.00   
Dacia Jogger              Station Wagon    -            4.00  23207.50   
Dacia Logan               Station Wagon     Front       2.00  10990.00   
Dacia Sandero             Compact          -            4.00  18015.00   
                          Off-Road/Pick-Up -            1.00  10950.00   
                          Sedan             Front       1.00  10980.00   
                                           -            3.00  12670.00   
Fiat 124 Spider           Convertible       Rear        1.00  24980.00   
                                           -            1.00  21960.00   
Fiat 500                  Compact           Front       5.00  17179.80   
                          Off-Road/Pick-Up  Front       1.00  29985.00   
                          Sedan            -            5.00  12285.80   
Fiat 500C                 Convertible       Front       7.00  17601.86   
                                           -            5.00  14957.60   
Fiat 500X                 Off-Road/Pick-Up  4WD         1.00  16990.00   
                                            Front       1.00  15350.00   
                          Sedan             Front       1.00  23990.00   
Fiat Punto                Compact          -            1.00   5799.00   
Fiat Tipo                 Sedan            -            4.00  17427.25   
                          Station Wagon     Front       1.00  31285.00   
                                           -            2.00  11730.00   
Ford Bronco               Off-Road/Pick-Up  4WD         1.00  47950.00   
Ford Ecosport             Compact          -            4.00  23865.00   
Ford Edge                 Off-Road/Pick-Up  4WD         4.00  34775.00   
Ford Explorer             Off-Road/Pick-Up  4WD         1.00  67990.00   
Ford Fiesta               Compact           Front      14.00  18719.21   
                                           -           22.00  14938.55   
                          Coupe            -            1.00  20740.00   
                          Sedan             Front       6.00  12950.83   
                                           -           37.00  17440.43   
Ford Focus                Coupe            -            2.00  26920.00   
                          Sedan             4WD         1.00  39980.00   
                                            Front       6.00  24807.50   
                                           -           30.00  25075.63   
                          Station Wagon     Front      11.00  24217.55   
                                           -           49.00  22602.02   
Ford Grand C              Station Wagon    -            1.00  19290.00   
Ford Grand Tourneo        Station Wagon    -            1.00  25180.00   
Ford Kuga                 Off-Road/Pick-Up  4WD         9.00  26435.56   
                                            Front       5.00  31054.00   
                                           -           39.00  29176.36   
Ford Mondeo               Sedan            -            3.00  22653.33   
                          Station Wagon     4WD         2.00  36465.00   
                                            Front       3.00  25223.33   
                                           -           14.00  24514.93   
Ford Mustang              Convertible       Rear        4.00  66215.00   
                                           -            6.00  46416.33   
                          Coupe             Rear       11.00  71872.73   
                                           -           13.00  45454.46   
Ford Puma                 Compact          -            1.00  24550.00   
                          Off-Road/Pick-Up -            8.0

Domain Knowledge demonstrates that the drivetrain of cars changes by their make_models and body_types most of the time. So it's decided to fill missing values with mode value of corresponding group.

In [104]:
df[df["body_type"] == "Off-Road/Pick-up"][["make_model", "body_type", "fuel_type", "drivetrain"]].value_counts()

Series([], dtype: int64)

In [105]:
# replace dash/hyphen sign "-" with np.nan

df["drivetrain"] = df["drivetrain"].replace('-', np.nan)

In [106]:
# fills the missing values in the "drivetrain" with the median value based on "make_model" and "body_type"

fill(df, "make_model", "body_type", "drivetrain", "mode")

COLUMN NAME    :  drivetrain
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  3
--------------------------------
 4WD       845
 Front    1362
 Rear      157
Name: drivetrain, dtype: int64


In [107]:
first_looking("empty_weight")

COLUMN NAME    :  empty_weight
--------------------------------
per_of_nulls   :  % 7.83
num_of_nulls   :  185
num_of_uniques :  587
--------------------------------
NaN        185
1280.00     39
1165.00     35
1733.00     35
1055.00     29
          ... 
1511.00      1
1439.00      1
1792.00      1
1992.00      1
1819.00      1
Name: empty_weight, Length: 588, dtype: int64




In [108]:
# All NaN values are converted to dash/hyphen sign "-" to see them in the output of describe() and explore the hidden pattern

df["empty_weight"].fillna("-", inplace=True)

In [109]:
# explore if there is any pattern among "make_model", "body_type", and "empty_weight" by "price"

df.groupby(["make_model", "body_type", "empty_weight"]).price.describe()

count     mean      std      min  \
make_model    body_type        empty_weight                                     
Dacia Duster  Off-Road/Pick-Up 1254.00        1.00 16490.00      NaN 16490.00   
                               1275.00        1.00 14990.00      NaN 14990.00   
                               1309.00        1.00 18990.00      NaN 18990.00   
                               1317.00        1.00 21150.00      NaN 21150.00   
                               1480.00        1.00 21690.00      NaN 21690.00   
...                                            ...      ...      ...      ...   
Volvo Xc90    Off-Road/Pick-Up 2327.00        2.00 97400.00     0.00 97400.00   
                               2328.00        4.00 84224.50 15271.49 70999.00   
                               2329.00        2.00 64990.00     0.00 64990.00   
                               2339.00        2.00 93890.00     0.00 93890.00   
                               2395.00        2.00 76930.00     0.00 76930.00   

                                                 25%      50%      75%  \
make_model    body_type        empty_weight                              
Dacia Duster  Off-Road/Pick-Up 1254.00      16490.00 16490.00 16490.00   
                               1275.00      14990.00 14990.00 14990.00   
                               1309.00      18990.00 18990.00 18990.00   
                               1317.00      21150.00 21150.00 21150.00   
                               1480.00      21690.00 21690.00 21690.00   
...                                              ...      ...      ...   
Volvo Xc90    Off-Road/Pick-Up 2327.00      97400.00 97400.00 97400.00   
                               2328.00      70999.00 84224.50 97450.00   
                               2329.00      64990.00 64990.00 64990.00   
                               2339.00      93890.00 93890.00 93890.00   
                               2395.00      76930.00 76930.00 76930.00   

                                                 max  
make_model    body_type        empty_weight           
Dacia Duster  Off-Road/Pick-Up 1254.00      16490.00  
                               1275.00      14990.00  
                               1309.00      18990.00  
                               1317.00      21150.00  
                               1480.00      21690.00  
...                                              ...  
Volvo Xc90    Off-Road/Pick-Up 2327.00      97400.00  
                               2328.00      97450.00  
                               2329.00      64990.00  
                               2339.00      93890.00  
                               2395.00      76930.00  

[1172 rows x 8 columns]

Domain Knowledge demonstrates that the empty weight of cars changes by their make_models and body_types most of the time. So it's decided to fill missing values with mode value of corresponding group.

In [110]:
# Let's replace dash/hyphen sign "-" with np.nan

df["empty_weight"].replace("-", np.nan, inplace=True)

In [111]:
# fills the missing values in the "drivetrain" with the mode value based on "make_model", "body_type", and "empty_weight"

fill(df, "make_model", "body_type", "empty_weight", "mode")

COLUMN NAME    :  empty_weight
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  587
--------------------------------
865.00     2
884.00     1
915.00     6
926.00     4
928.00     1
          ..
2513.00    1
2530.00    1
2560.00    2
2612.00    1
2690.00    1
Name: empty_weight, Length: 587, dtype: int64


In [112]:
first_looking("comfort_&_convenience")

COLUMN NAME    :  comfort_&_convenience
--------------------------------
per_of_nulls   :  % 0.13
num_of_nulls   :  3
num_of_uniques :  1735
--------------------------------
['Air conditioning, Armrest, Automatic climate control, 2 zones, Cruise control, Electrical side mirrors, Heated steering wheel, Hill Holder, Leather steering wheel, Light sensor, Lumbar support, Multi-function steering wheel, Navigation system, Parking assist system camera, Parking assist system sensors front, Parking assist system sensors rear, Power windows, Rain sensor, Seat heating, Split rear seats, Start-stop system, Tinted windows']                                                                                                                                                                                 12
['Air conditioning, Electrical side mirrors, Hill Holder, Leather steering wheel, Multi-function steering wheel, Power windows, Split rear seats, Tinted windows']                                        

In [113]:
# fills the missing values in the "comfort_&_convenience" with the mode value based on "make_model" and "body_type"

fill(df, "make_model", "body_type", "comfort_&_convenience", "mode")

COLUMN NAME    :  comfort_&_convenience
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  1735
--------------------------------
['360° camera, Air conditioning, Air suspension, Armrest, Automatic climate control, 2 zones, Auxiliary heating, Cruise control, Electric tailgate, Electrical side mirrors, Electrically adjustable seats, Heads-up display, Heated steering wheel, Hill Holder, Leather steering wheel, Light sensor, Lumbar support, Massage seats, Multi-function steering wheel, Navigation system, Panorama roof, Parking assist system camera, Parking assist system self-steering, Parking assist system sensors front, Parking assist system sensors rear, Power windows, Rain sensor, Seat heating, Seat ventilation, Split rear seats, Start-stop system, Sunroof, Tinted windows']    1
['360° camera, Air conditioning, Air suspension, Armrest, Automatic climate control, 2 zones, Cruise control, Electric tailgate, Electrical side mirrors, Electrically 

In [114]:
df['comfort_&_convenience'].unique()

array(["['360° camera, Air conditioning, Air suspension, Armrest, Automatic climate control, Auxiliary heating, Cruise control, Electric tailgate, Electrical side mirrors, Electrically adjustable seats, Heads-up display, Hill Holder, Keyless central door lock, Leather steering wheel, Light sensor, Lumbar support, Massage seats, Multi-function steering wheel, Navigation system, Parking assist system camera, Parking assist system self-steering, Parking assist system sensors front, Parking assist system sensors rear, Power windows, Rain sensor, Seat heating, Seat ventilation, Split rear seats, Start-stop system, Wind deflector']",
       "['Air conditioning, Armrest, Automatic climate control, Cruise control, Electrical side mirrors, Electrically adjustable seats, Hill Holder, Keyless central door lock, Leather steering wheel, Light sensor, Lumbar support, Multi-function steering wheel, Navigation system, Parking assist system camera, Parking assist system sensors front, Parking assist sy

In [115]:
premium = ["Electrical side mirrors", "Parking assist", "Air conditioning", "Hill Holder", "Power windows"]
premium_plus = ["Multi-function", "Navigation ", "Keyless central door lock", "Heads-up", "Massage seats", "heating", "Automatic climate control", "Heated"]

comfort_package = df['comfort_&_convenience'].apply(lambda sentence: "Premium Plus" if all(word in sentence for word in premium_plus) else ("Premium" if all(word in sentence for word in premium) else "Standard"))
comfort_package.value_counts()

Premium         1613
Standard         726
Premium Plus      25
Name: comfort_&_convenience, dtype: int64

Let us assign each car to related "Comfort & Conveniance Packages" defined above

In [116]:
premium = ["Electrical side mirrors", "Parking assist", "Air conditioning", "Hill Holder", "Power windows"]
premium_plus = ["Multi-function", "Navigation ", "Keyless central door lock", "Heads-up", "Massage seats", "heating", "Automatic climate control", "Heated"]

df['comfort_&_convenience_Package'] = df['comfort_&_convenience'].apply(lambda sentence: "Premium Plus" if all(word in sentence for word in premium_plus) else ("Premium" if all(word in sentence for word in premium) else "Standard"))
df

,make_model,make,model,location,price,body_type,type,doors,warranty,mileage,first_registration,gearbox,fuel_type,seller,seats,engine_size,gears,co_emissions,drivetrain,comfort_&_convenience,entertainment_&_media,safety_&_security,extras,empty_weight,full_service_history,upholstery,production_date,previous_owner,energy_efficiency_class,new,combined_fuel_cons,comfort_&_convenience_Package
0,Mercedes-Benz E 400,Mercedes-Benz,E 400,"Julius-Rütgers-Str. 1, 15537 Erkner, DE",64870.00,Convertible,Used,2.00,24,78473.00,01/2020,Automatic,Diesel (Particle filter),Dealer,4.00,2925.00,9.00,162.00,4WD,"['360° camera, Air conditioning, Air suspensio...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Ambient lighting, Automatically ...",2010.00,Yes,NaN,2020.00,1.00,efficient,Mercedes-Benz E 400,6.20,Premium
1,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"Wormser Landstraße 194, 67346 Speyer, DE",197500.00,Convertible,Employee's car,2.00,0,9550.00,04/2021,Automatic,Super 95,Dealer,2.00,3982.00,7.00,298.00,Rear,"['Air conditioning, Armrest, Automatic climate...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Alarm system, ...","Alloy wheels, Ambient lighting, Automatically ...",1735.00,Yes,NaN,2021.00,1.00,unefficient,Mercedes-Benz AMG GT,13.00,Premium
2,Mercedes-Benz C 400,Mercedes-Benz,C 400,"Autofocus 3, 15517 Fürstenwalde, DE",63370.00,Convertible,Employee's car,2.00,24,20145.00,01/2021,Automatic,Super 95,Dealer,4.00,2996.00,9.00,196.00,4WD,"['Armrest, Automatic climate control, 2 zones,...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Ambient lighting, Automatically ...",1825.00,Yes,NaN,2021.00,1.00,unefficient,Mercedes-Benz C 400,8.60,Standard
3,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"Meckenheimer Straße 12, 53359 Rheinbach, DE",175000.00,Convertible,Used,2.00,24,3561.00,07/2021,Automatic,Super 95,Dealer,2.00,3982.00,7.00,296.00,Rear,"['Air conditioning, Armrest, Automatic climate...","['Android Auto, Apple CarPlay, Digital radio, ...","['ABS, Blind spot monitor, Central door lock, ...","Alloy wheels, Automatically dimming interior m...",1700.00,No,NaN,2021.00,1.00,unefficient,Mercedes-Benz AMG GT,12.90,Standard
4,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"St.-Georgener-Straße 1, 79111 Freiburg, DE",149890.00,Convertible,Used,3.00,Yes,23851.00,12/2019,Automatic,Super E10 95 (Particle filter),Dealer,2.00,3982.00,7.00,262.00,Rear,"['Air conditioning, Armrest, Automatic climate...","['Apple CarPlay, Bluetooth, Digital radio, Han...","['ABS, Adaptive Cruise Control, Alarm system, ...","Alloy wheels (20""), Ambient lighting, Automati...",1700.00,Yes,Full leather,2019.00,2.00,unefficient,Mercedes-Benz AMG GT,11.50,Premium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,Volvo S60,Volvo,S60,"Lagerstraße 12, 93055 Regensburg, DE",44990.00,Sedan,Used,4.00,12,10500.00,06/2021,Automatic,Super 95,Dealer,5.00,1969.00,8.00,140.00,Front,"['Air conditioning, Automatic climate control,...","['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Sport seats, Sport suspension, T...",1752.00,Yes,Part leather,2021.00,1.00,efficient,Volvo S60,6.20,Premium
2360,Volvo S90,Volvo,S90,"Lagerstraße 12, 93055 Regensburg, DE",67990.00,Sedan,Demonstration,4.00,24,9990.00,02/2022,Automatic,Diesel (Particle filter),Dealer,5.00,1969.00,8.00,128.00,4WD,"['Air conditioning, Armrest, Automatic climate...","['Bluetooth, Digital cockpit, Digital radio, H...","['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Sport seats, Touch screen, Voice...",1790.00,No,Full leather,2021.00,1.00,efficient,Volvo S90,4.90,Premium
2361,Volvo S90,Volvo,S90,"Cannstatter Str. 46, 70190 Stuttgart, DE",37900.00,Sedan,Used,4.00,12,37846.00,04/2019,Automat

In [117]:
df['comfort_&_convenience_Package'].value_counts(dropna=False)

Premium         1613
Standard         726
Premium Plus      25
Name: comfort_&_convenience_Package, dtype: int64

In [118]:
df.shape

(2364, 32)

In [119]:
# There is no need for old 'comfort_&_convenience' since a new feature of 'comfort_&_convenience_Package' is created

df.drop("comfort_&_convenience", axis=1, inplace=True)

In [120]:
df.shape

(2364, 31)

In [121]:
first_looking("entertainment_&_media")

COLUMN NAME    :  entertainment_&_media
--------------------------------
per_of_nulls   :  % 0.21
num_of_nulls   :  5
num_of_uniques :  433
--------------------------------
['Android Auto, Apple CarPlay, Bluetooth, Digital radio, Hands-free equipment, Integrated music streaming, MP3, On-board computer, Radio, USB']                                                                                                        139
['Android Auto, Apple CarPlay, Bluetooth, Digital radio, Hands-free equipment, MP3, On-board computer, Radio, USB']                                                                                                                                    104
['Android Auto, Apple CarPlay, Bluetooth, Digital cockpit, Digital radio, Hands-free equipment, MP3, On-board computer, Radio, Sound system, USB']                                                                                                      71
['Android Auto, Apple CarPlay, Bluetooth, Digital radio, Hands-free equipm

In [122]:
# fills the missing values in the "entertainment_&_media" with the mode value based on "make_model" and "body_type"

fill(df, "make_model", "body_type", "entertainment_&_media", "mode")

COLUMN NAME    :  entertainment_&_media
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  433
--------------------------------
['Android Auto, Apple CarPlay, Bluetooth, CD player, Digital cockpit, Digital radio, Hands-free equipment, Induction charging for smartphones, Integrated music streaming, MP3, On-board computer, Radio, Sound system, USB']                            2
['Android Auto, Apple CarPlay, Bluetooth, CD player, Digital cockpit, Digital radio, Hands-free equipment, Induction charging for smartphones, Integrated music streaming, MP3, On-board computer, Radio, Sound system, USB, WLAN / WiFi hotspot']       3
['Android Auto, Apple CarPlay, Bluetooth, CD player, Digital cockpit, Digital radio, Hands-free equipment, Induction charging for smartphones, Integrated music streaming, MP3, On-board computer, Radio, USB, WLAN / WiFi hotspot']                     1
['Android Auto, Apple CarPlay, Bluetooth, CD player, Digital cockpit, Digit

In [123]:
df['entertainment_&_media'].unique()

array(["['Android Auto, Apple CarPlay, Bluetooth, Digital cockpit, Digital radio, Hands-free equipment, Induction charging for smartphones, Integrated music streaming, MP3, On-board computer, Radio, Sound system, USB']",
       "['Android Auto, Apple CarPlay, Bluetooth, Digital cockpit, Digital radio, Hands-free equipment, Integrated music streaming, MP3, On-board computer, Radio, Sound system, USB']",
       "['Android Auto, Apple CarPlay, Bluetooth, Digital radio, Hands-free equipment, Integrated music streaming, MP3, On-board computer, Radio, Sound system, USB']",
       "['Android Auto, Apple CarPlay, Digital radio, On-board computer, Radio, Sound system']",
       "['Apple CarPlay, Bluetooth, Digital radio, Hands-free equipment, On-board computer, Radio, Sound system, WLAN / WiFi hotspot']",
       "['Android Auto, Apple CarPlay, Bluetooth, Digital radio, Hands-free equipment, MP3, On-board computer, Radio, Sound system, USB, WLAN / WiFi hotspot']",
       "['Bluetooth, CD player,

In [124]:
#Option 1
media_plus = ["Digital", "Television"]

# media_plus = ["Digital radio", "Hands-free", "Television"]

entertainment_media_package = df['entertainment_&_media'].apply(lambda sentence: "Premium Media" if any(word in sentence for word in media_plus) else "Standard Media")
entertainment_media_package.value_counts()

Premium Media     1767
Standard Media     597
Name: entertainment_&_media, dtype: int64

In [125]:
#Option 2
# media_plus = ["Digital", "Television"]

media_plus = ["Digital radio", "Hands-free", "Television", "Induction charging for smartphones", "Apple"]

entertainment_media_package = df['entertainment_&_media'].apply(lambda sentence: "Premium Media" if any(word in sentence for word in media_plus) else "Standard Media")
entertainment_media_package.value_counts()

Premium Media     2296
Standard Media      68
Name: entertainment_&_media, dtype: int64

**Let us assign each car to related ``"Entertainment & Media Packages"`` defined above:**

In [126]:
media_plus = ["Digital", "Television"]

df['entertainment_&_media_Package'] = df['entertainment_&_media'].apply(lambda sentence: "Premium Media" if any(word in sentence for word in media_plus) else "Standard Media")
df

,make_model,make,model,location,price,body_type,type,doors,warranty,mileage,first_registration,gearbox,fuel_type,seller,seats,engine_size,gears,co_emissions,drivetrain,entertainment_&_media,safety_&_security,extras,empty_weight,full_service_history,upholstery,production_date,previous_owner,energy_efficiency_class,new,combined_fuel_cons,comfort_&_convenience_Package,entertainment_&_media_Package
0,Mercedes-Benz E 400,Mercedes-Benz,E 400,"Julius-Rütgers-Str. 1, 15537 Erkner, DE",64870.00,Convertible,Used,2.00,24,78473.00,01/2020,Automatic,Diesel (Particle filter),Dealer,4.00,2925.00,9.00,162.00,4WD,"['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Ambient lighting, Automatically ...",2010.00,Yes,NaN,2020.00,1.00,efficient,Mercedes-Benz E 400,6.20,Premium,Premium Media
1,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"Wormser Landstraße 194, 67346 Speyer, DE",197500.00,Convertible,Employee's car,2.00,0,9550.00,04/2021,Automatic,Super 95,Dealer,2.00,3982.00,7.00,298.00,Rear,"['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Alarm system, ...","Alloy wheels, Ambient lighting, Automatically ...",1735.00,Yes,NaN,2021.00,1.00,unefficient,Mercedes-Benz AMG GT,13.00,Premium,Premium Media
2,Mercedes-Benz C 400,Mercedes-Benz,C 400,"Autofocus 3, 15517 Fürstenwalde, DE",63370.00,Convertible,Employee's car,2.00,24,20145.00,01/2021,Automatic,Super 95,Dealer,4.00,2996.00,9.00,196.00,4WD,"['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Ambient lighting, Automatically ...",1825.00,Yes,NaN,2021.00,1.00,unefficient,Mercedes-Benz C 400,8.60,Standard,Premium Media
3,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"Meckenheimer Straße 12, 53359 Rheinbach, DE",175000.00,Convertible,Used,2.00,24,3561.00,07/2021,Automatic,Super 95,Dealer,2.00,3982.00,7.00,296.00,Rear,"['Android Auto, Apple CarPlay, Digital radio, ...","['ABS, Blind spot monitor, Central door lock, ...","Alloy wheels, Automatically dimming interior m...",1700.00,No,NaN,2021.00,1.00,unefficient,Mercedes-Benz AMG GT,12.90,Standard,Premium Media
4,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"St.-Georgener-Straße 1, 79111 Freiburg, DE",149890.00,Convertible,Used,3.00,Yes,23851.00,12/2019,Automatic,Super E10 95 (Particle filter),Dealer,2.00,3982.00,7.00,262.00,Rear,"['Apple CarPlay, Bluetooth, Digital radio, Han...","['ABS, Adaptive Cruise Control, Alarm system, ...","Alloy wheels (20""), Ambient lighting, Automati...",1700.00,Yes,Full leather,2019.00,2.00,unefficient,Mercedes-Benz AMG GT,11.50,Premium,Premium Media
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,Volvo S60,Volvo,S60,"Lagerstraße 12, 93055 Regensburg, DE",44990.00,Sedan,Used,4.00,12,10500.00,06/2021,Automatic,Super 95,Dealer,5.00,1969.00,8.00,140.00,Front,"['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Sport seats, Sport suspension, T...",1752.00,Yes,Part leather,2021.00,1.00,efficient,Volvo S60,6.20,Premium,Premium Media
2360,Volvo S90,Volvo,S90,"Lagerstraße 12, 93055 Regensburg, DE",67990.00,Sedan,Demonstration,4.00,24,9990.00,02/2022,Automatic,Diesel (Particle filter),Dealer,5.00,1969.00,8.00,128.00,4WD,"['Bluetooth, Digital cockpit, Digital radio, H...","['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Sport seats, Touch screen, Voice...",1790.00,No,Full leather,2021.00,1.00,efficient,Volvo S90,4.90,Premium,Premium Media
2361,Volvo S90,Volvo,S90,"Cannstatter Str. 46, 70190 Stuttgart, DE",37900.00,Sedan,Used,4.00,12,37846.00,04/2019,Automatic,Diesel (Particle filter),Dealer,5.00,1969.00,8.00,121.00,4WD,"['Android Auto, Apple CarPlay, Bluetooth, Digi...","['ABS, Adaptive Cruise Control, Blind spot mon...","Alloy wheels (19""), Ambient lighting, Automati...",1866.00,Yes,Part leather,2019.00,2.00

In [127]:
df['entertainment_&_media_Package'].value_counts(dropna=False)

Premium Media     1767
Standard Media     597
Name: entertainment_&_media_Package, dtype: int64

In [128]:
df.shape

(2364, 32)

In [129]:
# There is no need for old 'entertainment_&_media' since a new feature of 'entertainment_&_media_Package' is created

df.drop("entertainment_&_media", axis=1, inplace=True)

In [130]:
df.shape

(2364, 31)

In [131]:
first_looking("extras")

COLUMN NAME    :  extras
--------------------------------
per_of_nulls   :  % 0.21
num_of_nulls   :  5
num_of_uniques :  1523
--------------------------------
Alloy wheels, Catalytic Converter, Touch screen, Voice Control                                                                                                                                    44
Alloy wheels, Catalytic Converter, Roof rack, Touch screen                                                                                                                                        39
Alloy wheels, Catalytic Converter, Touch screen                                                                                                                                                   35
Alloy wheels, Catalytic Converter, Roof rack, Touch screen, Voice Control                                                                                                                         26
Alloy wheels, Catalytic Converter                    

In [132]:
#  fills the missing values in the "extras" with the mode value based on "make_model" and "body_type"

fill(df, "make_model", "body_type", "extras", "mode")

COLUMN NAME    :  extras
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  1523
--------------------------------
All season tyres, Alloy wheels (15"), Ambient lighting, Automatically dimming interior mirror, Cargo barrier, E10-enabled, Emergency tyre repair kit, Spare tyre, Spoiler, Sport seats, Touch screen, Voice Control                    1
All season tyres, Alloy wheels (15"), Ambient lighting, Automatically dimming interior mirror, Cargo barrier, E10-enabled, Emergency tyre repair kit, Spare tyre, Spoiler, Sport seats, Touch screen, Voice Control, Winter package    3
All season tyres, Alloy wheels (15"), Automatically dimming interior mirror, E10-enabled, Electronic parking brake, Emergency tyre                                                                                                     1
All season tyres, Alloy wheels (15"), Automatically dimming interior mirror, E10-enabled, Emergency tyre repair kit                            

In [133]:
first_looking("safety_&_security")

COLUMN NAME    :  safety_&_security
--------------------------------
per_of_nulls   :  % 0.13
num_of_nulls   :  3
num_of_uniques :  1373
--------------------------------
['ABS, Adaptive Cruise Control, Adaptive headlights, Blind spot monitor, Central door lock, Central door lock with remote control, Daytime running lights, Distance warning system, Driver drowsiness detection, Driver-side airbag, Electronic stability control, Emergency brake assistant, Emergency system, Fog lights, Full-LED headlights, Head airbag, High beam assist, Immobilizer, Isofix, LED Daytime Running Lights, LED Headlights, Lane departure warning system, Passenger-side airbag, Power steering, Rear airbag, Side airbag, Speed limit control system, Tire pressure monitoring system, Traction control, Traffic sign recognition']    63
['ABS, Adaptive Cruise Control, Adaptive headlights, Bi-Xenon headlights, Blind spot monitor, Central door lock, Daytime running lights, Distance warning system, Driver drowsiness detection

In [134]:
# fills the missing values in the "safety_&_security" with the mode value based on "make_model" and "body_type"

fill(df, "make_model", "body_type", "safety_&_security", "mode")

COLUMN NAME    :  safety_&_security
--------------------------------
per_of_nulls   :  % 0.0
num_of_nulls   :  0
num_of_uniques :  1373
--------------------------------
['ABS, Adaptive Cruise Control, Adaptive headlights, Alarm system, Bi-Xenon headlights, Blind spot monitor, Central door lock, Daytime running lights, Distance warning system, Driver drowsiness detection, Driver-side airbag, Electronic stability control, Emergency brake assistant, Emergency system, Fog lights, Full-LED headlights, Glare-free high beam headlights, High beam assist, Immobilizer, Isofix, LED Daytime Running Lights, LED Headlights, Lane departure warning system, Laser headlights, Passenger-side airbag, Power steering, Side airbag, Speed limit control system, Tire pressure monitoring system, Traction control, Traffic sign recognition, Xenon headlights']     1
['ABS, Adaptive Cruise Control, Adaptive headlights, Alarm system, Bi-Xenon headlights, Blind spot monitor, Central door lock, Daytime running lights, 

In [135]:
# display all unique values in "safety_&_security"
# pd.set_option('display.max_rows', None)

df['safety_&_security'].unique()

array(["['ABS, Adaptive Cruise Control, Adaptive headlights, Blind spot monitor, Central door lock, Daytime running lights, Distance warning system, Driver drowsiness detection, Driver-side airbag, Electronic stability control, Emergency system, Full-LED headlights, Glare-free high beam headlights, Head airbag, High beam assist, Immobilizer, Isofix, LED Daytime Running Lights, LED Headlights, Lane departure warning system, Passenger-side airbag, Power steering, Side airbag, Speed limit control system, Tire pressure monitoring system, Traction control, Traffic sign recognition']",
       "['ABS, Adaptive Cruise Control, Alarm system, Blind spot monitor, Central door lock, Daytime running lights, Distance warning system, Driver drowsiness detection, Driver-side airbag, Electronic stability control, Emergency brake assistant, Emergency system, Fog lights, Full-LED headlights, Glare-free high beam headlights, High beam assist, Immobilizer, Isofix, LED Daytime Running Lights, LED Headlights

In [136]:
premium = ["Tire pressure", "Daytime running lights", "LED Headlight", "Lane departure warning system"]
premium_plus = ["Traction control", "Emergency brake assistant", "Electronic stability control", "Adaptive"]

safety_security_package = df['safety_&_security'].apply(lambda sentence: "Safety Premium Package" if all(word in sentence for word in premium) else ("Safety Premium Plus Package" if all(word in sentence for word in premium_plus) else "Safety Standard Package"))
safety_security_package.value_counts()

Safety Premium Package         1240
Safety Standard Package         820
Safety Premium Plus Package     304
Name: safety_&_security, dtype: int64

In [137]:
premium = ["Tire pressure", "Daytime running lights", "LED Headlight"]
premium_plus = ["Traction control", "Emergency brake assistant", "Electronic stability control"]

df['safety_&_security_Package'] = df['safety_&_security'].apply(lambda sentence: "Safety Premium Package" if all(word in sentence for word in premium) else ("Safety Premium Plus Package" if all(word in sentence for word in premium_plus) else "Safety Standard Package"))
df

,make_model,make,model,location,price,body_type,type,doors,warranty,mileage,first_registration,gearbox,fuel_type,seller,seats,engine_size,gears,co_emissions,drivetrain,safety_&_security,extras,empty_weight,full_service_history,upholstery,production_date,previous_owner,energy_efficiency_class,new,combined_fuel_cons,comfort_&_convenience_Package,entertainment_&_media_Package,safety_&_security_Package
0,Mercedes-Benz E 400,Mercedes-Benz,E 400,"Julius-Rütgers-Str. 1, 15537 Erkner, DE",64870.00,Convertible,Used,2.00,24,78473.00,01/2020,Automatic,Diesel (Particle filter),Dealer,4.00,2925.00,9.00,162.00,4WD,"['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Ambient lighting, Automatically ...",2010.00,Yes,NaN,2020.00,1.00,efficient,Mercedes-Benz E 400,6.20,Premium,Premium Media,Safety Premium Package
1,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"Wormser Landstraße 194, 67346 Speyer, DE",197500.00,Convertible,Employee's car,2.00,0,9550.00,04/2021,Automatic,Super 95,Dealer,2.00,3982.00,7.00,298.00,Rear,"['ABS, Adaptive Cruise Control, Alarm system, ...","Alloy wheels, Ambient lighting, Automatically ...",1735.00,Yes,NaN,2021.00,1.00,unefficient,Mercedes-Benz AMG GT,13.00,Premium,Premium Media,Safety Premium Package
2,Mercedes-Benz C 400,Mercedes-Benz,C 400,"Autofocus 3, 15517 Fürstenwalde, DE",63370.00,Convertible,Employee's car,2.00,24,20145.00,01/2021,Automatic,Super 95,Dealer,4.00,2996.00,9.00,196.00,4WD,"['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Ambient lighting, Automatically ...",1825.00,Yes,NaN,2021.00,1.00,unefficient,Mercedes-Benz C 400,8.60,Standard,Premium Media,Safety Premium Package
3,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"Meckenheimer Straße 12, 53359 Rheinbach, DE",175000.00,Convertible,Used,2.00,24,3561.00,07/2021,Automatic,Super 95,Dealer,2.00,3982.00,7.00,296.00,Rear,"['ABS, Blind spot monitor, Central door lock, ...","Alloy wheels, Automatically dimming interior m...",1700.00,No,NaN,2021.00,1.00,unefficient,Mercedes-Benz AMG GT,12.90,Standard,Premium Media,Safety Premium Package
4,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"St.-Georgener-Straße 1, 79111 Freiburg, DE",149890.00,Convertible,Used,3.00,Yes,23851.00,12/2019,Automatic,Super E10 95 (Particle filter),Dealer,2.00,3982.00,7.00,262.00,Rear,"['ABS, Adaptive Cruise Control, Alarm system, ...","Alloy wheels (20""), Ambient lighting, Automati...",1700.00,Yes,Full leather,2019.00,2.00,unefficient,Mercedes-Benz AMG GT,11.50,Premium,Premium Media,Safety Premium Package
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,Volvo S60,Volvo,S60,"Lagerstraße 12, 93055 Regensburg, DE",44990.00,Sedan,Used,4.00,12,10500.00,06/2021,Automatic,Super 95,Dealer,5.00,1969.00,8.00,140.00,Front,"['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Sport seats, Sport suspension, T...",1752.00,Yes,Part leather,2021.00,1.00,efficient,Volvo S60,6.20,Premium,Premium Media,Safety Premium Package
2360,Volvo S90,Volvo,S90,"Lagerstraße 12, 93055 Regensburg, DE",67990.00,Sedan,Demonstration,4.00,24,9990.00,02/2022,Automatic,Diesel (Particle filter),Dealer,5.00,1969.00,8.00,128.00,4WD,"['ABS, Adaptive Cruise Control, Adaptive headl...","Alloy wheels, Sport seats, Touch screen, Voice...",1790.00,No,Full leather,2021.00,1.00,efficient,Volvo S90,4.90,Premium,Premium Media,Safety Premium Package
2361,Volvo S90,Volvo,S90,"Cannstatter Str. 46, 70190 Stuttgart, DE",37900.00,Sedan,Used,4.00,12,37846.00,04/2019,Automatic,Diesel (Particle filter),Dealer,5.00,1969.00,8.00,121.00,4WD,"['ABS, Adaptive Cruise Control, Blind spot mon...","Alloy wheels (19""), Ambient lighting, Automati...",1866.00,Yes,Part leather,2019.00,2.00,efficient,Volvo S90,4.60,Premium,Premium Media,Safety Premium Package
2362,Volvo S60,Volvo,S60,"Lagerstraße 12, 93055 Regensburg, DE",46890.00,Sedan,Demonstration,4.00,24,9990.00,02/2022,Automatic,Super 95,Dealer,5.00,1969.00,8.00,145.00,Front,"['ABS

In [138]:
df['safety_&_security_Package'].value_counts(dropna=False)

Safety Premium Package         1435
Safety Premium Plus Package     606
Safety Standard Package         323
Name: safety_&_security_Package, dtype: int64

In [139]:
df.shape

(2364, 32)

In [140]:
df.drop("safety_&_security", axis=1, inplace=True)

In [141]:
df.shape

(2364, 31)

FINAL

In [142]:
df.head()

,make_model,make,model,location,price,body_type,type,doors,warranty,mileage,first_registration,gearbox,fuel_type,seller,seats,engine_size,gears,co_emissions,drivetrain,extras,empty_weight,full_service_history,upholstery,production_date,previous_owner,energy_efficiency_class,new,combined_fuel_cons,comfort_&_convenience_Package,entertainment_&_media_Package,safety_&_security_Package
0,Mercedes-Benz E 400,Mercedes-Benz,E 400,"Julius-Rütgers-Str. 1, 15537 Erkner, DE",64870.00,Convertible,Used,2.00,24,78473.00,01/2020,Automatic,Diesel (Particle filter),Dealer,4.00,2925.00,9.00,162.00,4WD,"Alloy wheels, Ambient lighting, Automatically ...",2010.00,Yes,NaN,2020.00,1.00,efficient,Mercedes-Benz E 400,6.20,Premium,Premium Media,Safety Premium Package
1,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"Wormser Landstraße 194, 67346 Speyer, DE",197500.00,Convertible,Employee's car,2.00,0,9550.00,04/2021,Automatic,Super 95,Dealer,2.00,3982.00,7.00,298.00,Rear,"Alloy wheels, Ambient lighting, Automatically ...",1735.00,Yes,NaN,2021.00,1.00,unefficient,Mercedes-Benz AMG GT,13.00,Premium,Premium Media,Safety Premium Package
2,Mercedes-Benz C 400,Mercedes-Benz,C 400,"Autofocus 3, 15517 Fürstenwalde, DE",63370.00,Convertible,Employee's car,2.00,24,20145.00,01/2021,Automatic,Super 95,Dealer,4.00,2996.00,9.00,196.00,4WD,"Alloy wheels, Ambient lighting, Automatically ...",1825.00,Yes,NaN,2021.00,1.00,unefficient,Mercedes-Benz C 400,8.60,Standard,Premium Media,Safety Premium Package
3,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"Meckenheimer Straße 12, 53359 Rheinbach, DE",175000.00,Convertible,Used,2.00,24,3561.00,07/2021,Automatic,Super 95,Dealer,2.00,3982.00,7.00,296.00,Rear,"Alloy wheels, Automatically dimming interior m...",1700.00,No,NaN,2021.00,1.00,unefficient,Mercedes-Benz AMG GT,12.90,Standard,Premium Media,Safety Premium Package
4,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"St.-Georgener-Straße 1, 79111 Freiburg, DE",149890.00,Convertible,Used,3.00,Yes,23851.00,12/2019,Automatic,Super E10 95 (Particle filter),Dealer,2.00,3982.00,7.00,262.00,Rear,"Alloy wheels (20""), Ambient lighting, Automati...",1700.00,Yes,Full leather,2019.00,2.00,unefficient,Mercedes-Benz AMG GT,11.50,Premium,Premium Media,Safety Premium Package


In [143]:
df.shape

(2364, 31)

In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2364 entries, 0 to 2363
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   make_model                     2364 non-null   object 
 1   make                           2364 non-null   object 
 2   model                          2364 non-null   object 
 3   location                       2364 non-null   object 
 4   price                          2364 non-null   float64
 5   body_type                      2364 non-null   object 
 6   type                           2364 non-null   object 
 7   doors                          2364 non-null   float64
 8   warranty                       2364 non-null   object 
 9   mileage                        2364 non-null   float64
 10  first_registration             2364 non-null   object 
 11  gearbox                        2364 non-null   object 
 12  fuel_type                      2364 non-null   o

In [145]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

def check_obj_columns(df):
    '''
    Returns NO PROBLEM or column/s which has/have mixed object types.
    '''    
    
    tdf = df.select_dtypes(include=['object']).applymap(type)
    for col in tdf:
        if len(set(tdf[col].values)) > 1:
            print("Column" + " " + color.BOLD + color.RED + col + color.END + " " + "has mixed object types." )
    else:
        if len(set(tdf[col].values)) == 1:
            print(color.BOLD + color.GREEN + " " + "NO PROBLEM" + " " + color.END + " " + "with the data types of Columns in the DataFrame.")

check_obj_columns(df)

Column upholstery has mixed object types.
 NO PROBLEM  with the data types of Columns in the DataFrame.


In [146]:
df.isnull().sum()*100 / df.shape[0]

make_model                      0.00
make                            0.00
model                           0.00
location                        0.00
price                           0.00
body_type                       0.00
type                            0.00
doors                           0.00
warranty                        0.00
mileage                         0.00
first_registration              0.00
gearbox                         0.00
fuel_type                       0.00
seller                          0.00
seats                           0.00
engine_size                     0.00
gears                           0.00
co_emissions                    0.00
drivetrain                      0.00
extras                          0.00
empty_weight                    0.00
full_service_history            0.00
upholstery                      5.67
production_date                 0.00
previous_owner                  6.77
energy_efficiency_class         0.00
new                             0.00
c

In [147]:
df.duplicated(keep=False).sum()

327

In [148]:
df[df.duplicated(keep=False)]

,make_model,make,model,location,price,body_type,type,doors,warranty,mileage,first_registration,gearbox,fuel_type,seller,seats,engine_size,gears,co_emissions,drivetrain,extras,empty_weight,full_service_history,upholstery,production_date,previous_owner,energy_efficiency_class,new,combined_fuel_cons,comfort_&_convenience_Package,entertainment_&_media_Package,safety_&_security_Package
79,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"Julius-Rütgers-Str. 1, 15537 Erkner, DE",135670.00,Coupe,Used,5.00,24,74566.00,08/2019,Automatic,Super 95,Dealer,4.00,3982.00,9.00,257.00,4WD,"Alloy wheels, Ambient lighting, Automatically ...",2120.00,Yes,NaN,2019.00,1.00,unefficient,Mercedes-Benz AMG GT,11.30,Standard,Premium Media,Safety Premium Package
80,Mercedes-Benz Amg Gt,Mercedes-Benz,Amg Gt,"Julius-Rütgers-Str. 1, 15537 Erkner, DE",135670.00,Coupe,Used,5.00,24,74566.00,08/2019,Automatic,Super 95,Dealer,4.00,3982.00,9.00,257.00,4WD,"Alloy wheels, Ambient lighting, Automatically ...",2120.00,Yes,NaN,2019.00,1.00,unefficient,Mercedes-Benz AMG GT,11.30,Standard,Premium Media,Safety Premium Package
83,Mercedes-Benz S 63 Amg,Mercedes-Benz,S 63 Amg,"Wörth 17, 94034 Passau, DE",165850.00,Coupe,Used,3.00,24,26338.00,02/2020,Automatic,Regular/Benzine 91 (Particle filter),Dealer,4.00,3982.00,9.00,254.00,4WD,"Alloy wheels, Ambient lighting, Automatically ...",2080.00,Yes,Full leather,2020.00,1.00,unefficient,Mercedes-Benz S 63 AMG,11.20,Premium,Premium Media,Safety Premium Package
84,Mercedes-Benz S 63 Amg,Mercedes-Benz,S 63 Amg,"Wörth 17, 94034 Passau, DE",165850.00,Coupe,Used,3.00,24,26338.00,02/2020,Automatic,Regular/Benzine 91 (Particle filter),Dealer,4.00,3982.00,9.00,254.00,4WD,"Alloy wheels, Ambient lighting, Automatically ...",2080.00,Yes,Full leather,2020.00,1.00,unefficient,Mercedes-Benz S 63 AMG,11.20,Premium,Premium Media,Safety Premium Package
141,Mercedes-Benz C 43 Amg,Mercedes-Benz,C 43 Amg,"Altendorfer Straße 44 C, 45127 Essen, DE",67670.00,Coupe,Demonstration,2.00,24,4900.00,05/2022,Automatic,Regular/Benzine 91 (Particle filter),Dealer,4.00,2996.00,9.00,222.00,4WD,"Alloy wheels, Ambient lighting, Automatically ...",1750.00,Yes,Other,2022.00,1.00,unefficient,Mercedes-Benz C 43 AMG,9.70,Standard,Premium Media,Safety Premium Package
142,Mercedes-Benz C 43 Amg,Mercedes-Benz,C 43 Amg,"Altendorfer Straße 44 C, 45127 Essen, DE",67670.00,Coupe,Demonstration,2.00,24,4900.00,05/2022,Automatic,Regular/Benzine 91 (Particle filter),Dealer,4.00,2996.00,9.00,222.00,4WD,"Alloy wheels, Ambient lighting, Automatically ...",1750.00,Yes,Other,2022.00,1.00,unefficient,Mercedes-Benz C 43 AMG,9.70,Standard,Premium Media,Safety Premium Package
203,Mercedes-Benz G 400,Mercedes-Benz,G 400,"Hallesche Straße 150, 99734 Nordhausen, DE",164710.00,Off-Road/Pick-Up,Employee's car,5.00,24,23989.00,04/2021,Automatic,Diesel (Particle filter),Dealer,5.00,2925.00,6.00,281.00,4WD,"Alloy wheels, Ambient lighting, Cargo barrier,...",2472.00,Yes,Full leather,2021.00,1.00,unefficient,Mercedes-Benz G 400,10.70,Standard,Premium Media,Safety Premium Package
204,Mercedes-Benz G 400,Mercedes-Benz,G 400,"Hallesche Straße 150, 99734 Nordhausen, DE",164710.00,Off-Road/Pick-Up,Employee's car,5.00,24,23989.00,04/2021,Automatic,Diesel (Particle filter),Dealer,5.00,2925.00,6.00,281.00,4WD,"Alloy wheels, Ambient lighting, Cargo barrier,...",2472.00,Yes,Full leather,2021.00,1.00,unefficient,Mercedes-Benz G 400,10.70,Standard,Premium Media,Safety Premium Package
263,Mercedes-Benz C 200,Mercedes-Benz,C 200,"Halberstädter Chaussee 200, 39116 Magdeburg, DE",24989.00,Station Wagon,Used,5.00,12,86063.00,11/2017,Manual,Regular/Benzine 91,Dealer,5.00,1991.00,6.00,128.00,Rear,"Alloy wheels (18""), Cargo barrier, E10-enabled...",1495.00,Yes,Other,2017.00,1.00,efficient,Mercedes-Benz C 200,5.50,Premium,Standard Media,Safety Premium Plus Package
264,Mercedes-Benz C 200,Mercedes-Benz,C 200,"Halberstädter Chaussee 200, 39116 Magdeburg, DE",24989.00,Station Wagon,Used,5.00,12,86063.00,11/2017,Manual,Regular/Be

In [149]:
df.isnull().sum()

make_model                         0
make                               0
model                              0
location                           0
price                              0
body_type                          0
type                               0
doors                              0
warranty                           0
mileage                            0
first_registration                 0
gearbox                            0
fuel_type                          0
seller                             0
seats                              0
engine_size                        0
gears                              0
co_emissions                       0
drivetrain                         0
extras                             0
empty_weight                       0
full_service_history               0
upholstery                       134
production_date                    0
previous_owner                   160
energy_efficiency_class            0
new                                0
c

In [150]:
df.to_csv('autoscout_cleaned_filled.csv', index=False)